In [ ]:
import sys
sys.path.append(r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF')

import cvxpy as cp
import pandas as pd
import numpy as np
import math
import cvxpy_dcopf as cd
import cvxpy_scopf as cs
#import cvxpy_imb as ci
from cvxpylayers.torch import CvxpyLayer
import torch
import torchvision
import timeit
import precontingency as pc
import torch.nn as nn
import time

from tqdm import tqdm
import itertools
import torch.nn.functional as F
from torch import Tensor

from torch.utils.data import DataLoader
import torch.utils.data as Data
from sklearn.model_selection import train_test_split

import psutil

import loadsampling as ls
import pickle

#SOLVER SETTINGS FOR SCS
# used as solver for cvxpylayer in NN
# SCS is standard solver for CVXPYlayer, you can specify other solvers

SCS_solver_args={'use_indirect': False,
         'gpu': False,
         'verbose': False, # False
         'normalize': True, #True heuristic data rescaling
         'max_iters': 10000, #2500 giving the maximum number of iterations
         'scale': 100, #1 if normalized, rescales by this factor
         'eps':1e-3, #1e-3 convergence tolerance
         'cg_rate': 2, #2 for indirect, tolerance goes down like 1/iter^cg_rate
         'alpha': 1.5, #1.5 relaxation parameter
         'rho_x':1e-3, #1e-3 x equality constraint scaling
         'acceleration_lookback': 10, #10
         'write_data_filename':None}


SCS_solver_args2={'use_indirect': False,
         'gpu': False,
         'verbose': False, # False
         'normalize': True, #True heuristic data rescaling
         'max_iters': 20000, #2500 giving the maximum number of iterations
         'scale': 1, #1 if normalized, rescales by this factor
         'eps':1e-3, #1e-3 convergence tolerance
         'cg_rate': 2, #2 for indirect, tolerance goes down like 1/iter^cg_rate
         'alpha': 1.5, #1.5 relaxation parameter
         'rho_x':1e-3, #1e-3 x equality constraint scaling
         'acceleration_lookback': 10, #10
         'write_data_filename':None}


SCS_solver_args3={#'use_indirect': False,
         #'gpu': False,
         'verbose': False, # False
         'normalize': False, #True heuristic data rescaling
         'max_iters': 10000, #2500 giving the maximum number of iterations
         'scale': 1, #1 if normalized, rescales by this factor
         'eps':1e-3, #1e-3 convergence tolerance ????????
         #'cg_rate': 2, #2 for indirect, tolerance goes down like 1/iter^cg_rate ????????
         'alpha': 1.5, #1.5 relaxation parameter
         'rho_x':1e-3, #1e-3 x equality constraint scaling
         'acceleration_lookback': 10, #10
         'write_data_filename':None}

ECOS_solver_args = {"solve_method":"ECOS",
    'verbose': False, # False
    'max_iters': 50000, # Maximum number of iterations
    'reltol': 1e-3, # Convergence tolerance
    # Add other ECOS solver arguments as needed
}


data = pd.read_excel(r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\IEEE27mod.xlsx',sheet_name=None)
Sbase = data['par']['base'][0]
Nloads = len(data['load'])
load_loc = torch.from_numpy(np.array(data['load']['bus']))
Ngens = len(data['gen'])
gen_loc = torch.from_numpy(np.array(data['gen']['bus']))
Nlines = len(data['line'])
Nbus = len(data['bus'])
print(data["line"]['max_f'][0]/Sbase)

Nsamples = 1000
Niterations = 200
batch_size = 100
train = 0.8
tollerance_crit = 0.1
learning_rate = 0.01
wd = 0.1 

In [ ]:
from torch.nn.parallel import DataParallel

if torch.cuda.is_available():
    # GPU is available
    device = torch.device('cuda')
    print("GPU is available")
else:
    # GPU is not available
    device = torch.device('cpu')
    print("GPU is not available")


if torch.cuda.device_count() >= 1:
    device_ids = list(range(torch.cuda.device_count()))
else:
    device_ids = None
    
num_workers = torch.cuda.device_count()

In [ ]:
#data
#X = np.array(data['load']['p'])/Sbase*np.random.uniform(low=0.75, high=1.25, size = (Nsamples,Nloads))

#LB = 0.75*np.ones(Nloads)
#UB = 1.25*np.ones(Nloads)
#X = np.array(data['load']['p'])/Sbase*ls.kumaraswamymontecarlo(1.6, 2.8, 0.75, LB, UB, Nsamples).T

file_path = r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\trained models\training data\Xtrain.pkl' 
with open(file_path, 'rb') as f:
    X = pickle.load(f)

Xtrain = torch.tensor(X[0:int(Nsamples),:],dtype = torch.float32)        
Xtrain_transpose = Xtrain.transpose(0,1)
#Xtest = torch.tensor(X[int(train*Nsamples):,:],dtype = torch.float64)
#standardise
Xmin, Xmax,Xmean,Xstd = np.min(X, axis = 0),np.max(X, axis = 0),np.mean(X, axis = 0),np.std(X, axis = 0)
Xscal = torch.tensor(( X - Xmean ) / Xstd, dtype=torch.float32)
#Xscaltrain = Xscal[0:int(train*Nsamples),:]
#Xscaltest = Xscal[int(train*Nsamples):,:]


c_max = np.max(np.array(data['gen']['cost']))
c_min = np.min(np.array(data['gen']['cost']))

gencost = torch.tensor(np.array(data['gen']['cost']),dtype = torch.float32)  
c_delta = c_max - c_min
cgi0 = torch.tensor((np.array(data['gen']['cost'])-c_min)/c_delta,dtype = torch.float32)
alpha = 1

PTDF = torch.from_numpy(pc.compptdfs(data)).to(torch.float32)

In [ ]:
start = timeit.default_timer()
#problem0, Pd0, cg0, cll0, Pgi, Fl0, Th0, lam10, lam20 = cd.create_dcopf_lower(data)
problem0, Pd0, cll0, Pgi, Fl0, Th0, lam10, lam20 = cd.create_dcopf_correction(data)
assert problem0.is_dpp()

#cvxpylayer0 = CvxpyLayer(problem0, parameters=[Pd0, cg0, cll0], variables=[Pgi, Fl0, Th0, lam10, lam20])
cvxpylayer0 = CvxpyLayer(problem0, parameters=[Pd0, cll0], variables=[Pgi, Fl0, Th0, lam10, lam20])
time1 = timeit.default_timer() - start
print("Time to create model: ", time1)

max_f = data["line"]['max_f'][0]/Sbase
l = len(data['line'])
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)


In [ ]:
import pickle
dict_method = 'pickle'

start_load = time.time()

if dict_method == 'pickle': 
    file_path = r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\LODF_list\IEEE27\LODF_listN1.pkl'  
    with open(file_path, 'rb') as f:
        LODF_dict_N1 = pickle.load(f)

Ncontingencies_N1 = int(l) 
tot_lodf_batches_N1 = len(LODF_dict_N1)
lodf_batch_size_N1 = Ncontingencies_N1
last_batch_size_N1 = Ncontingencies_N1 - (lodf_batch_size_N1*(tot_lodf_batches_N1-1))

if dict_method == 'pickle': 
    file_path = r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\LODF_list\IEEE27\LODF_listN2.pkl'  
    with open(file_path, 'rb') as f:
        LODF_dict_N2 = pickle.load(f)

Ncontingencies_N2 = int(l*(l-1)/2)
tot_lodf_batches_N2 = len(LODF_dict_N2)
lodf_batch_size_N2 = Ncontingencies_N2
last_batch_size_N2 = Ncontingencies_N2 - (lodf_batch_size_N2*(tot_lodf_batches_N2-1))

if dict_method == 'pickle': 
    file_path = r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\LODF_list\IEEE27\LODF_listN3.pkl'  
    with open(file_path, 'rb') as f:
        LODF_dict_N3 = pickle.load(f)

Ncontingencies_N3 = int(l*(l-1)/2*(l-2)/3)
tot_lodf_batches_N3 = len(LODF_dict_N3)
lodf_batch_size_N3 = Ncontingencies_N3 #20000
last_batch_size_N3 = Ncontingencies_N3 - (lodf_batch_size_N3*(tot_lodf_batches_N3-1))

if dict_method == 'pickle': 
    file_path = r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\LODF_list\IEEE27\LODF_listN4.pkl'  
    with open(file_path, 'rb') as f:
        LODF_dict_N4 = pickle.load(f)

Ncontingencies_N4 = int(l*(l-1)/2*(l-2)/3*(l-3)/4)
tot_lodf_batches_N4 = len(LODF_dict_N4)
lodf_batch_size_N4 = 20000
last_batch_size_N4 = Ncontingencies_N4 - (lodf_batch_size_N4*(tot_lodf_batches_N4-1))

if dict_method == 'pickle': 
    file_path = r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\LODF_list\IEEE27\LODF_listN5.pkl'  
    with open(file_path, 'rb') as f:
        LODF_dict_N5 = pickle.load(f)

Ncontingencies_N5 = int(l*(l-1)/2*(l-2)/3*(l-3)/4*(l-4)/5)
tot_lodf_batches_N5 = len(LODF_dict_N5)
lodf_batch_size_N5 = 20000 # Ncontingencies #20000
last_batch_size_N5 = Ncontingencies_N5 - (lodf_batch_size_N5*(tot_lodf_batches_N5-1))
        
    
end_load = time.time()

print('Load LODF time:', end_load - start_load)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)




In [ ]:
scenario = 'basecase'

if scenario == 'basecase':
    #===================== Base case probabilties
    file_path = 'probabilities/IEEE39/probabilities_N1.pkl'
    with open(file_path, 'rb') as f:
        probabilities_N1 = pickle.load(f)

    file_path = 'probabilities/IEEE39/probabilities_N2.pkl'
    with open(file_path, 'rb') as f:
        probabilities_N2 = pickle.load(f)

    file_path = 'probabilities/IEEE39/probabilities_N3.pkl'
    with open(file_path, 'rb') as f:
        probabilities_N3 = pickle.load(f)

elif scenario == 'stormy1':
#===================== Stormy node 1 probabilties
    file_path = 'probabilities/IEEE39/Storm/probabilities_N1_bigstorm2.pkl'
    with open(file_path, 'rb') as f:
        probabilities_N1 = pickle.load(f)

    file_path = 'probabilities/IEEE39/Storm/probabilities_N2_bigstorm2.pkl'
    with open(file_path, 'rb') as f:
        probabilities_N2 = pickle.load(f)

    file_path = 'probabilities/IEEE39/Storm/probabilities_N3_bigstorm2.pkl'
    with open(file_path, 'rb') as f:
        probabilities_N3 = pickle.load(f)
        
file_path = 'probabilities/IEEE39/line_lengths.pkl'
with open(file_path, 'rb') as f:
    line_len = pickle.load(f)
    

line_lengths = torch.tensor(line_len).unsqueeze(1)
print(torch.mean(line_lengths))
    
print(probabilities_N1.sum()/l, probabilities_N1.shape, probabilities_N1.max(), probabilities_N1.min())
print(probabilities_N2.sum()/l, probabilities_N2.shape, probabilities_N2.max(), probabilities_N2.min())
print(probabilities_N3.sum()/l, probabilities_N3.shape, probabilities_N3.max(), probabilities_N3.min())


In [ ]:
#Benchmark either DCOPF or SCOPF
benchmark = 'SCOPF' #'SCOPF'

Pgisscopf = torch.zeros(Ngens,int(Nsamples),dtype = torch.float32)
Fl0sscopf = torch.zeros(Nlines,int(Nsamples),dtype = torch.float32)
Flcs = torch.zeros(Nlines,l,int(Nsamples),dtype = torch.float32)
lam1s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
lam2s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
lam1sc = torch.zeros(Nbus,l,int(Nsamples),dtype = torch.float32)
lam2sc = torch.zeros(Nbus,l,int(Nsamples),dtype = torch.float32)
Th0 = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)

#======== remove islanding cases from contingency list
lcontingencies = range(l)
islanding_cases = [15,20]
lcontingencies_filtered = [l for l in lcontingencies if l not in islanding_cases]
lcontingencies = lcontingencies_filtered

time_scopfs = time.time()

#======== determine infeasible cases
# add loads
load_profile = torch.zeros((Nsamples,Nbus)).to(torch.float32)
load_profile[:, load_loc] = torch.from_numpy(X).to(torch.float32)

if benchmark == 'SCOPF':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf(data, lcontingencies)

elif benchmark == 'SCOPF2':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc = cs.create_scopf2(data, lcontingencies)

elif benchmark == 'DCOPF': 
    #problem0scopf, Pd0scopf, cg0scopf, cl0scopf, Pgiscopf, Fl0scopf, Th0scopf, lam1dc, lam2dc = cd.create_dcopf_upper(data)
    problem0scopf, Pd0scopf, cg0scopf, cll0scopf, Pgiscopf, Fl0scopf, Th0scopf, lam1, lam2 = cd.create_dcopf_lower(data)
    cll0scopf.value = np.zeros(Ngens)

cg0scopf.value = np.array(data['gen']['cost'])
for entry in tqdm(range(int(Nsamples)),position=0, leave=True):   
    Pd0scopf.value = X[entry,:] #np.array(data['load']['p'])/Sbase    
    solution  = problem0scopf.solve(solver=cp.ECOS)    # cp.MOSEK
    if problem0scopf.status in ["infeasible", "unbounded"]:
         # Otherwise, problem.value is inf or -inf, respectively.
         print("Problem infeasible or unbounded")
         #print("Optimal value: %s" % problem0scopf.value)
    Pgisscopf[:,entry] = torch.tensor(Pgiscopf.value)
    Fl0sscopf[:,entry] = torch.tensor(Fl0scopf.value)
    #Flcs[:,:,entry] = insert_zero_columns(torch.tensor(Flc.value),islanding_cases)
    Th0[:,entry] = torch.tensor(Th0scopf.value)
    
    lam1s[:,entry] = torch.tensor(lam1.value)
    lam2s[:,entry] = torch.tensor(lam2.value)
    #lam1sc[:,:,entry] = insert_zero_columns(torch.tensor(lam1c.value),islanding_cases)
    #lam2sc[:,:,entry] = insert_zero_columns(torch.tensor(lam2c.value),islanding_cases)
    
time_scopfs = time.time() - time_scopfs
cost_scopf = torch.matmul(gencost,Pgisscopf)

# add gens
load_profile[:, gen_loc] += Pgisscopf.T

# add smallest values to zero elements
nonzero_values, _ = torch.min(load_profile.masked_fill(load_profile == 0, float('inf')), dim=1, keepdim=True)
mask = load_profile == 0
load_profile = torch.where(mask, nonzero_values, load_profile)


In [ ]:
with torch.set_grad_enabled(True):
    N, D_in, D_out = Nsamples, Nloads, Ngens
    H1, H2, H3 = 8,8,8
   
    #very large network
    model = torch.nn.Sequential(
        torch.nn.Linear(D_in, H1),
        torch.nn.ReLU(),
        nn.Dropout(p=0.2),
        torch.nn.Linear(H1, H2),
        torch.nn.ReLU(),
        nn.Dropout(p=0.2),
        torch.nn.Linear(H2, H3),
        torch.nn.ReLU(),
        nn.Dropout(p=0.2),
        torch.nn.Linear(H3, D_out), #before 23/12 when everything was working!
        #torch.nn.Sigmoid(),
        torch.nn.Tanh(),
        #torch.nn.Softmax(dim=0),
    )
    
    #model = torch.nn.RNN(input_size = D_in, hidden_size = D_out, num_layers=3, nonlinearity = 'tanh', batch_first = True)
    model.double()
    
    def init_weights(u):
        if type(u) == nn.Linear:
            torch.nn.init.xavier_uniform_(u.weight)
            #u.reset_parameters() # default gaussion initializaiton
            torch.nn.init.xavier_uniform_(u.weight, gain=nn.init.calculate_gain('relu'))
            u.bias.data.fill_(0)    
    model.apply(init_weights)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    # adaptive momentum with weight decay
    
    model.to(torch.float32)
    print(model)

print('Load LODF time:', end_load - start_load)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    
num_params_full = sum(p.numel() for p in model.parameters())
#num_params_TT = sum(p.numel() for p in modelTT.parameters())
print('')
print('FC neural network:', num_params_full)
#print('TT neural network:', num_params_TT)
print('input loads:', D_in, 'output gens:', D_out)

In [ ]:
if device_ids:
    model = DataParallel(model, device_ids=device_ids)
    model = model.to(device)
    print(device_ids)

model.to(torch.float32)
print(next(model.parameters()).device)
print(next(model.parameters()).dtype)

# prompt to empty GPU cache: torch.cuda.empty_cache()

In [ ]:
gen_indices = data['gen']['bus'] 
generator_nodes = np.zeros(Nbus)
for index in gen_indices:
    generator_nodes[index] = 1
gen_nodes = np.nonzero(generator_nodes)
gen_nodes = gen_nodes[0].tolist()
print(gen_nodes)

load_indices = data['load']['bus'] 
load_nodes = np.zeros(Nbus)
for index in load_indices:
    load_nodes[index] = 1
load_nodes = np.nonzero(load_nodes)
load_nodes = load_nodes[0].tolist()
print(load_nodes)

In [ ]:
start = time.time()
max_p = torch.from_numpy(np.array(-data["gen"]['min_p']/Sbase))
min_p = torch.from_numpy(np.array(data["gen"]['max_p']/Sbase))

#============== Initialize training testing data
Xscal_train, Xscal_test, cost_scopf_train, cost_scopf_test = train_test_split(Xscal, cost_scopf, test_size=1-train, random_state=42)

training_dataset = Data.TensorDataset(Xscal_train, cost_scopf_train)
testing_dataset = Data.TensorDataset(Xscal_test, cost_scopf_test)

train_loader = Data.DataLoader(dataset=training_dataset, batch_size=batch_size, shuffle=True)
test_loader = Data.DataLoader(dataset=testing_dataset, batch_size=batch_size, shuffle=False)  # Note: shuffle=False for testing

import torch.nn.functional as F

flow_penalty = 1000
crit_penalty = 0
infeasibility_penalty = 0
cost_penalty = 1
soft_penalty = 100
Niterations = 10


#================ Initialize tensors
error = np.zeros(shape=(Niterations,6))
exp_inf = np.zeros(shape=(Niterations,6))
avg_infeasibility = np.zeros(shape=(Niterations,2))
relcost = np.zeros(shape=(Niterations,2))

time_CF = np.zeros(shape=(Niterations,1))
memory_CF = np.zeros(shape=(Niterations,1))
percentage_CF = np.zeros(shape=(Niterations,1))
backward_CF = np.zeros(shape=(Niterations,1))

Pgis = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
Flis = torch.zeros(size = (Nlines,int(batch_size)),dtype = torch.float32)
clil = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
infeasibility = torch.zeros(int(batch_size), requires_grad = False)

relcotrain_epoch = torch.zeros(size = (int(Nsamples/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)
relcotest_epoch = torch.zeros(size = (int(Nsamples/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)

imbalancetrain_epoch = torch.zeros(size = (int(Nsamples*train/batch_size),3),dtype = torch.float32, requires_grad = False)
imbalancetest_epoch = torch.zeros(size = (round((Nsamples*(1-train))/batch_size),3),dtype = torch.float32, requires_grad = False)

expected_viol_epoch = torch.zeros(size = (int(Nsamples*train/batch_size),3),dtype = torch.float32, requires_grad = False)
expected_viol_test_epoch = torch.zeros(size = (round((Nsamples*(1-train))/batch_size),3),dtype = torch.float32, requires_grad = False)

infeasibilitytrain_epoch = torch.zeros(size = (int(Nsamples/batch_size),Nbus),dtype = torch.float32, requires_grad = False)
infeasibilitytest_epoch = torch.zeros(size = (int(Nsamples/batch_size),Nbus),dtype = torch.float32, requires_grad = False)
infeasibility_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

line_violation_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
crit_violation_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
avg_violation_list = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
count_violation_list = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

violation_percentage = torch.zeros(size = (int(Nsamples*train/batch_size),1),dtype = torch.float32, requires_grad = False)
time_contflow = torch.zeros(size = (int(Nsamples*train/batch_size),1),dtype = torch.float32, requires_grad = False)
memory_contflow = torch.zeros(size = (int(Nsamples*train/batch_size),1),dtype = torch.float32, requires_grad = False)
backward_contflow = torch.zeros(size = (int(Nsamples*train/batch_size),1),dtype = torch.float32, requires_grad = False)

#===== N-1
crit_violation_count_N1 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
line_violation_count_N1 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

#===== N-2
crit_violation_count_N2 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
line_violation_count_N2 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

#===== N-3
crit_violation_count_N3 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
line_violation_count_N3 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)


print("It |   Time   |    Imbalance    | Mod out  |   Infeasibility   |  Rel cost")
for epoch in range(Niterations):
    expected_infeasible = 0
    expected_infeasible_test = 0
    
    time_nnSCS = time.time()
    for i, (Xbatch, cost_scopf_batch) in enumerate(tqdm(train_loader)):
            start_forward = time.time()
            start_forward_memory = psutil.virtual_memory()[3] / 1000000000            
            
            #============= initialize
            model.train()
            optimizer.zero_grad() 
            model.zero_grad()
            Xbatch = Xbatch.clone().detach().to(device)
            cost_scopf_batch = cost_scopf_batch.clone().detach().to(device)

            Xtrain_batch = ((Xbatch.cpu() * Xstd) + Xmean).to(torch.float32).to(device)
            load_profile = torch.zeros((batch_size, Nbus)).to(device)
            load_profile[:, load_loc] = Xtrain_batch

            #================== Perform forward pass
            start_FP = time.time()
            clil = (model(Xbatch) + 1) / 2
            end_FP = time.time()
            
            #============= soft violation base case
            Pg_guess = torch.mul(max_p,clil) 
            
            Pg_train = torch.zeros((batch_size,Nbus)).to(torch.float32)
            Pd_train = torch.zeros((batch_size,Nbus)).to(torch.float32)

            Pg_train[:,gen_nodes] = Pg_guess.to(torch.float32)
            Pd_train[:,load_nodes] = Xtrain_batch.to(torch.float32)
                
            Fl_base = F.relu(torch.abs(PTDF@(Pg_train-Pd_train).T)-max_f).sum()/int(batch_size)
            
            #============= cvxpylayer
            start_cvxpy = time.time()
            #Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtrain_batch, gencost, clil, solver_args=ECOS_solver_args)
            Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtrain_batch, clil, solver_args=ECOS_solver_args)
            Pgis = Pgi.T
            Flis = Fli.T.to(torch.float32)
            end_cvxpy = time.time()
            
            #=============== modify load profile with generator outputs
            load_profile[:, gen_loc] += Pgi
            nonzero_values, _ = torch.min(load_profile.masked_fill(load_profile == 0, float('inf')), dim=1, keepdim=True)
            mask = load_profile == 0
            load_profile = torch.where(mask, nonzero_values, load_profile)

            #================ infeasibility
            infeasibility_tensor = (lam1i + lam2i)
            infeasibility_loss = infeasibility_tensor.sum() / int(batch_size)
            infeasibility_batch = infeasibility_tensor.sum(dim=0).detach()

            infeasibility_tensor[abs(infeasibility_tensor) < 1e-5] = 0
            infeasible_buses = (abs(infeasibility_tensor) > 1e-5)

            infeasible_cases = torch.zeros((batch_size, 1)).to(device)
            row_has_true = torch.any(infeasible_buses, axis=1)
            infeasible_cases[row_has_true] = True
            infeasible_cases = infeasible_cases.sum()

            #==================== cost
            batch_cost = torch.matmul(gencost, Pgis)
            batch_relco = ((batch_cost - cost_scopf_batch) / cost_scopf_batch) * 100
            relco_avg = torch.mean(batch_relco)

            cost_loss = torch.mean(batch_cost) #/ int(batch_size)

            #===================== compute contingency flows for each sample  
            imbalancetrain_batch_N1 = 0
            imbalancetrain_batch_N2 = 0
            imbalancetrain_batch_N3 = 0
            imbalancetrain_batch = 0            
            
            count_crit_violation_N1 = 0
            count_line_violation_N1 = 0
            count_violation_N1 = 0
            
            count_crit_violation_N2 = 0
            count_line_violation_N2 = 0
            count_violation_N2 = 0
            
            count_crit_violation_N3 = 0
            count_line_violation_N3 = 0
            count_violation_N3 = 0
            
            expected_flows = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = True)
            expected_flows_N1 = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = True)
            expected_flows_N2 = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = True)
            expected_flows_N3 = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = True)

            #============ N-1
            for b in (range(tot_lodf_batches_N1)):
                #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
                lodf_batch = lodf_batch_size_N1
                if b == (tot_lodf_batches_N1 - 1): 
                    lodf_batch = last_batch_size_N1
                left_ones = torch.ones((lodf_batch,1))
                right_ones = torch.ones((int(l*batch_size),1))
                Fli_0 = Flis.expand(lodf_batch, -1, -1)
                
                if b != (tot_lodf_batches_N1 - 1): 
                    start_prob = b*lodf_batch*l
                    end_prob = (b+1)*lodf_batch*l
                    P_outage = probabilities_N1[start_prob:end_prob]
                if b == (tot_lodf_batches_N1 - 1): 
                    end_prob = Ncontingencies_N1*l
                    start_prob = (Ncontingencies_N1*l) - (last_batch_size_N1*l)
                    P_outage = probabilities_N1[-(last_batch_size_N1*l):]*1000
                                
                index1 = LODF_dict_N1[b][0].detach()
                index2 = LODF_dict_N1[b][1].detach()
                indices = torch.stack((index1,index2)).detach()
                values = LODF_dict_N1[b][2].detach()
                shape = (l,int(l*lodf_batch))
                lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

                imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, batch_size))) - max_f)).detach().to(device)
                count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > tollerance_crit * max_f, dim=1)
                count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
                imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)

                #========= count the number of rows that have all zeros
                mask = torch.all(imbalancetrain_batch_loop == 0, dim=1) 
                indices_remove = mask.nonzero().squeeze(1)
                num_zero_rows = torch.sum(mask)
                
                indices_tot = torch.arange(start_prob, end_prob)
                mask_keep = ~torch.isin(indices_tot, indices_remove)
                indices_keep = indices_tot[mask_keep]
                indices_loc = indices_keep % l

                #============ create new lodf
                mask_flow = torch.ones(imbalancetrain_batch_loop.shape[0], dtype=torch.bool).to(device)
                mask_flow[indices_remove] = False
                lodf_test = lodf.to_dense()[mask_flow]
                lodf_test = lodf_test.clone().detach()
                lodf = lodf_test.to_sparse_coo().requires_grad_().to(torch.float32) 
                P_outage = P_outage[mask_flow]
                
                Fli_0 = Flis.repeat(lodf_batch, 1).to(torch.float32).to(device)[mask_flow]
                imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0+torch.sparse.mm(lodf,Flis))- max_f))
                count_violation_loop = torch.count_nonzero(imbalancetrain_batch_loop)#/int(batch_size) 
                expected_violation = torch.mul(P_outage,imbalancetrain_batch_loop).to(torch.float32)
                
                imbalancetrain_batch_N1 += torch.sum(imbalancetrain_batch_loop)/int(batch_size)
                expected_flows_N1 = torch.index_add(expected_flows_N1, 0, indices_loc.long(), expected_violation)
                
                count_violation_N1 += count_violation_loop
                count_crit_violation_N1 += torch.count_nonzero(count_crit_violation_loop.detach())
                count_line_violation_N1 += torch.count_nonzero(count_line_violation_loop.detach())
                
            #============ N-2
            for b in (range(tot_lodf_batches_N2)):
                #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
                lodf_batch = lodf_batch_size_N2
                if b == (tot_lodf_batches_N2 - 1): 
                    lodf_batch = last_batch_size_N2
                left_ones = torch.ones((lodf_batch,1))
                right_ones = torch.ones((int(l*batch_size),1))
                Fli_0 = Flis.expand(lodf_batch, -1, -1)
                
                if b != (tot_lodf_batches_N2 - 1): 
                    start_prob = b*lodf_batch*l
                    end_prob = (b+1)*lodf_batch*l
                    P_outage = probabilities_N2[start_prob:end_prob]
                if b == (tot_lodf_batches_N2 - 1): 
                    end_prob = Ncontingencies_N2*l
                    start_prob = (Ncontingencies_N2*l) - (last_batch_size_N2*l)
                    P_outage = probabilities_N2[-(last_batch_size_N2*l):]*1000
                                
                index1 = LODF_dict_N2[b][0].detach()
                index2 = LODF_dict_N2[b][1].detach()
                indices = torch.stack((index1,index2)).detach()
                values = LODF_dict_N2[b][2].detach()
                shape = (l,int(l*lodf_batch))
                lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

                imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, batch_size))) - max_f)).detach().to(device)
                count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > tollerance_crit * max_f, dim=1)
                count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
                imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)

                #========= count the number of rows that have all zeros
                mask = torch.all(imbalancetrain_batch_loop == 0, dim=1) 
                indices_remove = mask.nonzero().squeeze(1)
                num_zero_rows = torch.sum(mask)
                
                indices_tot = torch.arange(start_prob, end_prob)
                mask_keep = ~torch.isin(indices_tot, indices_remove)
                indices_keep = indices_tot[mask_keep]
                indices_loc = indices_keep % l

                #============ create new lodf
                mask_flow = torch.ones(imbalancetrain_batch_loop.shape[0], dtype=torch.bool).to(device)
                mask_flow[indices_remove] = False
                lodf_test = lodf.to_dense()[mask_flow]
                lodf_test = lodf_test.clone().detach()
                lodf = lodf_test.to_sparse_coo().requires_grad_().to(torch.float32) 
                P_outage = P_outage[mask_flow]
                
                Fli_0 = Flis.repeat(lodf_batch, 1).to(torch.float32).to(device)[mask_flow]
                imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0+torch.sparse.mm(lodf,Flis))- max_f))
                count_violation_loop = torch.count_nonzero(imbalancetrain_batch_loop)#/int(batch_size) 
                expected_violation = torch.mul(P_outage,imbalancetrain_batch_loop).to(torch.float32)
                
                imbalancetrain_batch_N2 += torch.sum(imbalancetrain_batch_loop)/int(batch_size)
                expected_flows_N2 = torch.index_add(expected_flows_N2, 0, indices_loc.long(), expected_violation)
                
                count_violation_N2 += count_violation_loop
                count_crit_violation_N2 += torch.count_nonzero(count_crit_violation_loop.detach())
                count_line_violation_N2 += torch.count_nonzero(count_line_violation_loop.detach())
                
                
            #============ N-3
            for b in (range(tot_lodf_batches_N3)):
                #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
                lodf_batch = lodf_batch_size_N3
                if b == (tot_lodf_batches_N3 - 1): 
                    lodf_batch = last_batch_size_N3
                left_ones = torch.ones((lodf_batch,1))
                right_ones = torch.ones((int(l*batch_size),1))
                Fli_0 = Flis.expand(lodf_batch, -1, -1)
                
                if b != (tot_lodf_batches_N3 - 1): 
                    start_prob = b*lodf_batch*l
                    end_prob = (b+1)*lodf_batch*l
                    P_outage = probabilities_N3[start_prob:end_prob]
                if b == (tot_lodf_batches_N3 - 1): 
                    end_prob = Ncontingencies_N3*l
                    start_prob = (Ncontingencies_N3*l) - (last_batch_size_N3*l)
                    P_outage = probabilities_N3[-(last_batch_size_N3*l):]*1000
                                
                index1 = LODF_dict_N3[b][0].detach()
                index2 = LODF_dict_N3[b][1].detach()
                indices = torch.stack((index1,index2)).detach()
                values = LODF_dict_N3[b][2].detach()
                shape = (l,int(l*lodf_batch))
                lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

                imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, batch_size))) - max_f)).detach().to(device)
                count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > tollerance_crit * max_f, dim=1)
                count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
                imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)

                #========= count the number of rows that have all zeros
                mask = torch.all(imbalancetrain_batch_loop == 0, dim=1) 
                indices_remove = mask.nonzero().squeeze(1)
                num_zero_rows = torch.sum(mask)
                
                indices_tot = torch.arange(start_prob, end_prob)
                mask_keep = ~torch.isin(indices_tot, indices_remove)
                indices_keep = indices_tot[mask_keep]
                indices_loc = indices_keep % l

                #============ create new lodf
                mask_flow = torch.ones(imbalancetrain_batch_loop.shape[0], dtype=torch.bool).to(device)
                mask_flow[indices_remove] = False
                lodf_test = lodf.to_dense()[mask_flow]
                lodf_test = lodf_test.clone().detach()
                lodf = lodf_test.to_sparse_coo().requires_grad_().to(torch.float32) 
                P_outage = P_outage[mask_flow]
                
                Fli_0 = Flis.repeat(lodf_batch, 1).to(torch.float32).to(device)[mask_flow]
                imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0+torch.sparse.mm(lodf,Flis))- max_f))

                count_violation_loop = torch.count_nonzero(imbalancetrain_batch_loop)#/int(batch_size) 
                expected_violation = torch.mul(P_outage,imbalancetrain_batch_loop).to(torch.float32)
                
                imbalancetrain_batch_N3 += torch.sum(imbalancetrain_batch_loop)/int(batch_size)
                expected_flows_N3 = torch.index_add(expected_flows_N3, 0, indices_loc.long(), expected_violation)
                
                count_violation_N3 += count_violation_loop
                count_crit_violation_N3 += torch.count_nonzero(count_crit_violation_loop.detach())
                count_line_violation_N3 += torch.count_nonzero(count_line_violation_loop.detach())
            
            expected_inf_N1 = torch.sum(expected_flows_N1)/int(batch_size)
            expected_inf_N2 = torch.sum(expected_flows_N2)/int(batch_size)
            expected_inf_N3 = torch.sum(expected_flows_N3)/int(batch_size)

            expected_infeasible_N1 = (torch.count_nonzero(expected_flows_N1)/torch.numel(expected_flows_N1))*100
            expected_infeasible_N2 = (torch.count_nonzero(expected_flows_N2)/torch.numel(expected_flows_N2))*100
            expected_infeasible_N3 = (torch.count_nonzero(expected_flows_N3)/torch.numel(expected_flows_N3))*100
            torch.set_printoptions(precision=9)
            
            imbalancetrain_batch = expected_inf_N1+expected_inf_N2+expected_inf_N3
            #imbalancetrain_batch_N1 + imbalancetrain_batch_N2 + imbalancetrain_batch_N3
            #print(imbalancetrain_batch_N1, imbalancetrain_batch_N3, imbalancetrain_batch_N3)
            
            #======================== Perform back pass
            loss = flow_penalty * imbalancetrain_batch + infeasibility_penalty * infeasibility_loss + cost_penalty * cost_loss + Fl_base * soft_penalty

            start_BP = time.time()
            loss.backward()
            end_BP = time.time()
            backward_batch = end_BP - start_BP
            #print('Backward pass:', end_BP - start_BP)
            
            nn.utils.clip_grad_norm_(model.parameters(), 0.5)

            optimizer.step()

            relcotrain_epoch[i,:] = batch_relco.detach()
            imbalancetrain_epoch[i,:] = torch.tensor([expected_inf_N1.detach(),expected_inf_N2.detach(),expected_inf_N3.detach()])
            expected_viol_epoch[i,:] = torch.tensor([expected_infeasible_N1.detach(),expected_infeasible_N2.detach(),expected_infeasible_N3.detach()])
            infeasibilitytrain_epoch[i,:] = infeasibility_batch.detach()
            infeasibility_count[i,:] = infeasible_cases.detach()
        
            #time_contflow[i,:] = time_contflow_batch
            #memory_contflow[i,:] = memory_contflow_batch
            #backward_contflow[i,:] = backward_batch
            
            #====== N-1
            crit_violation_count_N1[i,:] = count_crit_violation_N1.detach()
            line_violation_count_N1[i,:] = count_line_violation_N1.detach()
            
            #====== N-2
            crit_violation_count_N2[i,:] = count_crit_violation_N2.detach()
            line_violation_count_N2[i,:] = count_line_violation_N2.detach()
            
            #====== N-3
            crit_violation_count_N3[i,:] = count_crit_violation_N3.detach()
            line_violation_count_N3[i,:] = count_line_violation_N3.detach()
    
    with torch.no_grad():
        for i, (Xbatch, cost_scopf_batch) in enumerate(tqdm(test_loader)):
                
                model.eval() # deactivate dropout

                Xbatch = Xbatch.clone().detach().to(device)
                cost_scopf_batch = cost_scopf_batch.clone().detach().to(device)

                Xtest_batch = ((Xbatch.cpu() * Xstd) + Xmean).to(torch.float32).to(device)

                #================== Perform forward pass
                clil = (model(Xbatch) + 1) / 2

                #Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtest_batch, gencost, clil, solver_args=ECOS_solver_args)
                Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtest_batch, clil, solver_args=ECOS_solver_args)
                Pgis = Pgi.T
                Flis = Fli.T.to(torch.float32)

                #================ compute cost and infeasibility
                infeasibility_batch = torch.sum(lam1i) + torch.sum(lam2i)

                batch_cost = torch.matmul(gencost, Pgis)
                batch_relco = ((batch_cost - cost_scopf_batch) / cost_scopf_batch) * 100

                #===================== compute contingency flows for each sample
                imbalancetest_batch_N1 = 0
                imbalancetest_batch_N2 = 0
                imbalancetest_batch_N3 = 0
                imbalancetest_batch = 0
                
                expected_flows = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = False)
                expected_flows_N1 = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = False)
                expected_flows_N2 = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = False)
                expected_flows_N3 = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = False)

                #============ N-1
                for b in (range(tot_lodf_batches_N1)):
                    #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
                    lodf_batch = lodf_batch_size_N1
                    if b == (tot_lodf_batches_N1 - 1): 
                        lodf_batch = last_batch_size_N1
                    left_ones = torch.ones((lodf_batch,1))
                    right_ones = torch.ones((int(l*batch_size),1))
                    Fli_0 = Flis.expand(lodf_batch, -1, -1)
                    
                    if b != (tot_lodf_batches_N1 - 1): 
                        start_prob = b*lodf_batch*l
                        end_prob = (b+1)*lodf_batch*l
                        P_outage = probabilities_N1[start_prob:end_prob]
                    if b == (tot_lodf_batches_N1 - 1): 
                        end_prob = Ncontingencies_N1*l
                        start_prob = (Ncontingencies_N1*l) - (last_batch_size_N1*l)
                        P_outage = probabilities_N1[-(last_batch_size_N1*l):]

                    index1 = LODF_dict_N1[b][0].detach()
                    index2 = LODF_dict_N1[b][1].detach()
                    indices = torch.stack((index1,index2)).detach()
                    values = LODF_dict_N1[b][2].detach()
                    shape = (l,int(l*lodf_batch))
                    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

                    imbalancetest_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf,Flis).reshape((lodf_batch,l,batch_size)))- max_f)).detach()
                    imbalancetest_batch_loop = imbalancetest_batch_loop.reshape(int(l*lodf_batch), -1)
                    expected_flows_N1 += torch.mul(P_outage,imbalancetest_batch_loop).to(torch.float32).reshape((lodf_batch,l,batch_size)).sum(dim=0)

                    imbalancetest_batch_N1 += torch.sum(imbalancetest_batch_loop)/int(batch_size)

                #============ N-2
                for b in (range(tot_lodf_batches_N2)):
                    #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
                    lodf_batch = lodf_batch_size_N2
                    if b == (tot_lodf_batches_N2 - 1): 
                        lodf_batch = last_batch_size_N2
                    left_ones = torch.ones((lodf_batch,1))
                    right_ones = torch.ones((int(l*batch_size),1))
                    Fli_0 = Flis.expand(lodf_batch, -1, -1)
                    
                    if b != (tot_lodf_batches_N2 - 1): 
                        start_prob = b*lodf_batch*l
                        end_prob = (b+1)*lodf_batch*l
                        P_outage = probabilities_N2[start_prob:end_prob]
                    if b == (tot_lodf_batches_N2 - 1): 
                        end_prob = Ncontingencies_N2*l
                        start_prob = (Ncontingencies_N2*l) - (last_batch_size_N2*l)
                        P_outage = probabilities_N2[-(last_batch_size_N2*l):]

                    index1 = LODF_dict_N2[b][0].detach()
                    index2 = LODF_dict_N2[b][1].detach()
                    indices = torch.stack((index1,index2)).detach()
                    values = LODF_dict_N2[b][2].detach()
                    shape = (l,int(l*lodf_batch))
                    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

                    imbalancetest_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf,Flis).reshape((lodf_batch,l,batch_size)))- max_f)).detach()
                    imbalancetest_batch_loop = imbalancetest_batch_loop.reshape(int(l*lodf_batch), -1)
                    expected_flows_N2 += torch.mul(P_outage,imbalancetest_batch_loop).to(torch.float32).reshape((lodf_batch,l,batch_size)).sum(dim=0)

                    imbalancetest_batch_N2 += torch.sum(imbalancetest_batch_loop)/int(batch_size)


                #============ N-3
                for b in (range(tot_lodf_batches_N3)):
                    #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
                    lodf_batch = lodf_batch_size_N3
                    if b == (tot_lodf_batches_N3 - 1): 
                        lodf_batch = last_batch_size_N3
                    left_ones = torch.ones((lodf_batch,1))
                    right_ones = torch.ones((int(l*batch_size),1))
                    Fli_0 = Flis.expand(lodf_batch, -1, -1)
                    
                    if b != (tot_lodf_batches_N3 - 1): 
                        start_prob = b*lodf_batch*l
                        end_prob = (b+1)*lodf_batch*l
                        P_outage = probabilities_N3[start_prob:end_prob]
                    if b == (tot_lodf_batches_N3 - 1): 
                        end_prob = Ncontingencies_N3*l
                        start_prob = (Ncontingencies_N3*l) - (last_batch_size_N3*l)
                        P_outage = probabilities_N3[-(last_batch_size_N3*l):]

                    index1 = LODF_dict_N3[b][0].detach()
                    index2 = LODF_dict_N3[b][1].detach()
                    indices = torch.stack((index1,index2)).detach()
                    values = LODF_dict_N3[b][2].detach()
                    shape = (l,int(l*lodf_batch))
                    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

                    imbalancetest_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf,Flis).reshape((lodf_batch,l,batch_size)))- max_f)).detach()
                    imbalancetest_batch_loop = imbalancetest_batch_loop.reshape(int(l*lodf_batch), -1)
                    expected_flows_N3 += torch.mul(P_outage,imbalancetest_batch_loop).to(torch.float32).reshape((lodf_batch,l,batch_size)).sum(dim=0)

                    imbalancetest_batch_N3 += torch.sum(imbalancetest_batch_loop)/int(batch_size) 
                
                expected_inf_N1 = torch.sum(expected_flows_N1)/int(batch_size)
                expected_inf_N2 = torch.sum(expected_flows_N2)/int(batch_size)
                expected_inf_N3 = torch.sum(expected_flows_N3)/int(batch_size)

                expected_infeasible_N1 = (torch.count_nonzero(expected_flows_N1)/torch.numel(expected_flows_N1))*100
                expected_infeasible_N2 = (torch.count_nonzero(expected_flows_N2)/torch.numel(expected_flows_N2))*100
                expected_infeasible_N3 = (torch.count_nonzero(expected_flows_N3)/torch.numel(expected_flows_N3))*100
                torch.set_printoptions(precision=9)

                imbalancetest_batch = expected_inf_N1+expected_inf_N2+expected_inf_N3
                #imbalancetest_batch_N1 + imbalancetest_batch_N2 + imbalancetest_batch_N3
        
                relcotest_epoch[i,:] = batch_relco.detach()
                imbalancetest_epoch[i,:] = torch.tensor([expected_inf_N1.detach(),expected_inf_N2.detach(),expected_inf_N3.detach()])
                expected_viol_test_epoch[i,:] = torch.tensor([expected_infeasible_N1.detach(),expected_infeasible_N2.detach(),expected_infeasible_N3.detach()])
                infeasibilitytest_epoch[i,:] = infeasibility_batch.detach()
            
    time_nnSCS = time.time() - time_nnSCS  

    imbalancetrain = torch.sum(imbalancetrain_epoch, dim=0).detach()
    imbalancetest = torch.sum(imbalancetest_epoch, dim=0).detach()
    
    expected_violations_train = torch.mean(expected_viol_epoch, dim=0).detach()
    expected_violations_test = torch.mean(expected_viol_test_epoch, dim=0).detach()
    
    infeasibilitytrain = torch.mean(infeasibilitytrain_epoch).detach()
    infeasibilitytest = torch.mean(infeasibilitytest_epoch).detach()
    
    relcotrain = torch.mean(relcotrain_epoch).detach()
    relcotest = torch.mean(relcotest_epoch).detach()
    
    #======== N-1
    crit_violation_tot_N1 = torch.sum(crit_violation_count_N1).detach()
    line_violation_tot_N1 = torch.sum(line_violation_count_N1).detach()
    
    #======== N-1
    crit_violation_tot_N2 = torch.sum(crit_violation_count_N2).detach()
    line_violation_tot_N2 = torch.sum(line_violation_count_N2).detach()
    
    #======== N-1
    crit_violation_tot_N3 = torch.sum(crit_violation_count_N3).detach()
    line_violation_tot_N3 = torch.sum(line_violation_count_N3).detach()
    
    #====== convergence criteria
    infeasible_cases_tot = torch.sum(infeasibility_count)
    
    tot_violation_percentage = torch.mean(violation_percentage)
    tot_time_contflow = torch.sum(time_contflow)
    tot_memory_contflow = torch.mean(memory_contflow)
    tot_backward_contflow = torch.sum(backward_contflow)
    
    #============= 
    error[epoch,:] = torch.hstack((imbalancetrain,imbalancetest)).detach().numpy()
    exp_inf[epoch,:] = torch.hstack((expected_violations_train, expected_violations_test)).detach().numpy()
    avg_infeasibility[epoch,:] =  torch.stack((infeasibilitytrain,infeasibilitytest)).detach().numpy()
    relcost[epoch,:] =  torch.stack((relcotrain,relcotest)).detach().numpy()
    
    #================= computational graph
    time_CF[epoch,:] = tot_time_contflow
    memory_CF[epoch,:] = tot_memory_contflow 
    percentage_CF[epoch,:] = tot_violation_percentage
    backward_CF[epoch,:] = tot_backward_contflow

    print(epoch, " | ", round(time_nnSCS,0), " | ",np.round(error[epoch,:],3), " | ",np.round(exp_inf[epoch,:],3), " | ", np.round(avg_infeasibility[epoch,:],3), " | ", np.round(relcost[epoch,:], 3) )
    
    #======================= Convergence check infeasible cases
    print('current infeasibility:', (infeasible_cases_tot/int(train*Nsamples))*100, '%', 'goal:', 1, '%')
    
    print('')
    #print('N1 infeasible contingency cases:', line_violation_tot_N1, (line_violation_tot_N1/(Ncontingencies_N1*Nsamples))*100, '%')
    #print('N1 10%+ contingency cases:', crit_violation_tot_N1, (crit_violation_tot_N1/(Ncontingencies_N1*Nsamples))*100, '%')
    print('N1 expected avg violation:', error[epoch,3])
    
    print('')
    #print('N2 infeasible contingency cases:', line_violation_tot_N2, (line_violation_tot_N2/(Ncontingencies_N2*Nsamples))*100, '%')
    #print('N2 10%+ contingency cases:', crit_violation_tot_N2, (crit_violation_tot_N2/(Ncontingencies_N2*Nsamples))*100, '%')
    print('N2 expected avg violation:', error[epoch,4])
    
    print('')
    #print('N3 infeasible contingency cases:', line_violation_tot_N3, (line_violation_tot_N3/(Ncontingencies_N3*Nsamples))*100, '%')
    #print('N3 10%+ contingency cases:', crit_violation_tot_N3, (crit_violation_tot_N3/(Ncontingencies_N3*Nsamples))*100, '%')
    print('N3 expected avg violation:', error[epoch,5])
    print('')

    print('flow penalty:', flow_penalty*imbalancetrain_batch)
    print('infeasible penalty:', infeasibility_penalty*infeasibility_loss)
    print('cost penalty:', cost_penalty*cost_loss)
    print('soft loss:', soft_penalty*Fl_base)
    #print('extra;', loss_term)
    
        
end = time.time()
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)



In [ ]:
#file_path = r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\trained models\Nk models\ieee39_Nk_25k_1_soft.pt'
file_path = r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\trained models\Nk models\ieee39_new_prob.pt'


# Save the model
#torch.save(model.state_dict(), file_path)

In [ ]:
#file_path = r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\trained models\Nk models\ieee39_Nk_25000_1.pt'

# Load the saved model state dictionary
model.load_state_dict(torch.load(file_path))

# Set the model to evaluation mode
model.eval()
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

In [ ]:
#totaltime = start - end
#print("total time:", totaltime)

import matplotlib.pyplot as plt
plt.plot(error[:l, 0], 'r', label='N-1 train')
plt.plot(error[:l, 1], 'g', label='N-2 train')
plt.plot(error[:l, 2], 'b', label='N-3 train')

# Plot the N-1 test, N-2 test, and N-3 test columns
plt.plot(error[:l, 3], 'r--', label='N-1 test')
plt.plot(error[:l, 4], 'g--', label='N-2 test')
plt.plot(error[:l, 5], 'b--', label='N-3 test')
#new_list = 10*range(0, 10,l+1)
#plt.xticks(new_list)
plt.ylabel('Errors')
plt.xlabel('Iterations')
plt.legend()
plt.show()
plt.close()

plt.plot(exp_inf[:l, 0], 'r', label='N-1 train')
plt.plot(exp_inf[:l, 1], 'g', label='N-2 train')
plt.plot(exp_inf[:l, 2], 'b', label='N-3 train')

# Plot the N-1 test, N-2 test, and N-3 test columns
plt.plot(exp_inf[:l, 3], 'r--', label='N-1 test')
plt.plot(exp_inf[:l, 4], 'g--', label='N-2 test')
plt.plot(exp_inf[:l, 5], 'b--', label='N-3 test')
#new_list = 10*range(0, 10,l+1)
#plt.xticks(new_list)
plt.ylabel('Percentage')
plt.xlabel('Iterations')
plt.legend()
plt.show()
plt.close()


In [ ]:
Nsamples = 1000
#X = np.array(data['load']['p'])/Sbase*ls.kumaraswamymontecarlo(1.6, 2.8, 0.75, LB, UB, Nsamples).T
file_path = r'C:\Users\bgiraud\Documents\Thesis Bastien\I - Coding\01 - learning SCOPFs\01 - GLODF\trained models\training data\Xtest1.pkl' 
with open(file_path, 'rb') as f:
    X = pickle.load(f)
Xtrain = torch.tensor(X[0:int(Nsamples),:],dtype = torch.float32)        
Xtrain_transpose = Xtrain.transpose(0,1)
#Xtest = torch.tensor(X[int(train*Nsamples):,:],dtype = torch.float64)
#standardise
Xmin, Xmax,Xmean,Xstd = np.min(X, axis = 0),np.max(X, axis = 0),np.mean(X, axis = 0),np.std(X, axis = 0)
Xscal = torch.tensor(( X - Xmean ) / Xstd, dtype=torch.float32)

frequency = 0.0051
repair_time = 44
line_avg = torch.mean(line_lengths)

In [ ]:
#=============== Benchmark #1 contingency screening N-k
benchmark = 'risk_based' #'SCOPF'

if benchmark == 'SCOPF':
    lcontingencies = range(l)
    islanding_cases = [15,20]
    lcontingencies_filtered = [l for l in lcontingencies if l not in islanding_cases]
    lcontingencies = lcontingencies_filtered
    N1_contingencies = lcontingencies
elif benchmark == 'SCOPF1screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),1)))
elif benchmark == 'SCOPF2screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),2)))
elif benchmark == 'SCOPF3screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),3)))
elif benchmark == 'risk_based':
    N1_contingencies = np.array(list(itertools.combinations(range(Nlines),1)))
    N2_contingencies = np.array(list(itertools.combinations(range(Nlines),2)))

Pgisscopf = torch.zeros(Ngens,int(Nsamples),dtype = torch.float32)
Fl0sscopf = torch.zeros(Nlines,int(Nsamples),dtype = torch.float32)
#Flcs = torch.zeros(Nlines,len(Nk_contingencies),int(Nsamples),dtype = torch.float32)
lam1s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
lam2s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
lam1sc = torch.zeros(Nbus,52,int(Nsamples),dtype = torch.float32)
lam2sc = torch.zeros(Nbus,52,int(Nsamples),dtype = torch.float32)
Th0 = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)

#======== remove islanding cases from contingency list
if benchmark != 'SCOPF':
    lcontingencies = [0]
    l2contingencies = [1]
screening_iterations = 2

time_scopfs = time.time()

for i in range(screening_iterations):
    imbalance_list1 = []
    imbalance_list2 = []
    
    N1_zeros = list(range(len(N1_contingencies)))
    zero_indices = [l for l in N1_zeros if l not in lcontingencies]
    
    islanding_cases = [15,20]
    lcontingencies_filtered = [l for l in lcontingencies if l not in islanding_cases]
    lcontingencies = lcontingencies_filtered
    print(lcontingencies)
    
    if benchmark != 'SCOPF':
        islanding_cases2 = [0, 12, 13, 14, 19, 43, 44, 45, 50, 75, 80, 104, 109, 113, 117, 118, 132, 137, 159, 164, 167\
                            , 185, 190, 210, 215, 234, 239, 257, 262, 276, 279, 284, 300, 301, 305, 320, 325, 338, 339, 344,\
                            357, 362, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 395,\
                            408, 409, 410, 423, 424, 437, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 465,\
                            466, 467, 475, 476, 477, 495, 496, 500, 501, 507, 518, 525, 526, 527]
        lcontingencies_filtered2 = [l for l in l2contingencies if l not in islanding_cases2]
        l2contingencies = lcontingencies_filtered2
        print(l2contingencies)
    

    if benchmark == 'SCOPF':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf(data, lcontingencies)
    elif benchmark == 'SCOPF1screening':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf1_screening(data, lcontingencies)
    elif benchmark == 'SCOPF2screening':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf2_screening(data, lcontingencies)
    elif benchmark == 'SCOPF3screening':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf3_screening(data, lcontingencies)
    elif benchmark == 'risk_based':
        problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf_risk_based(data, lcontingencies, l2contingencies)
    

    cg0scopf.value = np.array(data['gen']['cost'])
    for entry in tqdm(range(int(Nsamples)),position=0, leave=True):   
        Pd0scopf.value = X[entry,:] #np.array(data['load']['p'])/Sbase    
        try:
            solution  = problem0scopf.solve(solver=cp.ECOS)    # cp.MOSEK
            if problem0scopf.status in ["infeasible", "unbounded"]:
                # Otherwise, problem.value is inf or -inf, respectively.
                print("Problem infeasible or unbounded")
                #print("Optimal value: %s" % problem0scopf.value)
            Pgisscopf[:,entry] = torch.tensor(Pgiscopf.value)
            Fl0sscopf[:,entry] = torch.tensor(Fl0scopf.value)
            Th0[:,entry] = torch.tensor(Th0scopf.value)

            lam1s[:,entry] = torch.tensor(lam1.value)
            lam2s[:,entry] = torch.tensor(lam2.value)
            if i != 0:
                lam1sc[:,:,entry] = torch.tensor(lam1c.value)
                lam2sc[:,:,entry] = torch.tensor(lam2c.value)
        except cp.error.SolverError:
            Pgisscopf[:,entry] = Pgisscopf[:,(entry-1)]
            Fl0sscopf[:,entry] = Fl0sscopf[:,(entry-1)]
            Th0[:,entry] = Th0[:,(entry-1)]

            lam1s[:,entry] = lam1s[:,(entry-1)]
            lam2s[:,entry] = lam2s[:,(entry-1)]
            lam1sc[:,:,entry] = lam1sc[:,:,(entry-1)]
            lam2sc[:,:,entry] = lam1sc[:,:,(entry-1)]
            
            print("Solver unable to solve for entry:", entry)
            continue
        
    
    cost_scopf = torch.matmul(gencost,Pgisscopf)
    
    if benchmark == 'SCOPF':
        break
        
    if i == (screening_iterations - 1):
        break

    #============ compute N1 contingencies
    Flis = Fl0sscopf
                
    for b in (range(tot_lodf_batches_N1)):
        lodf_batch = lodf_batch_size_N1
        if b == (tot_lodf_batches_N1 - 1):
            lodf_batch = last_batch_size_N1
        left_ones = torch.ones((lodf_batch,1))
        right_ones = torch.ones((int(l*batch_size),1))
        Fli_0 = Flis.expand(lodf_batch, -1, -1)

        index1 = LODF_dict_N1[b][0].detach()
        index2 = LODF_dict_N1[b][1].detach()
        indices = torch.stack((index1,index2)).detach()
        values = LODF_dict_N1[b][2].detach()
        shape = (l,int(l*lodf_batch))
        lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

        imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf,Flis).reshape((lodf_batch,l,Nsamples)))- max_f)).detach()

        #======= get indices of contingencies of highest violations
        summed_tensor = torch.sum(torch.sum(imbalancetrain_batch_loop, dim=1), dim=1)

        if b == 0:
            desired_length = len(summed_tensor)

        padding_length = desired_length - len(summed_tensor)
        padded_tensor = F.pad(summed_tensor, (0, padding_length), value=0)
        imbalance_list1.extend(padded_tensor)


    indices_per_iteration = 32 # len(summed_tensor)

    stacked_imbalance = torch.stack(imbalance_list1, dim=0)
    highest_indices = (np.argsort(stacked_imbalance.cpu())[-indices_per_iteration:]).tolist()
    new_indices = [idx for idx in highest_indices if idx not in lcontingencies] # Filter out the indices that are already in the list
    lcontingencies.extend(new_indices) # Append the new indices to the list
        
    if benchmark == 'risk_based': 
    #============ compute N2 contingencies
    
        for b in (range(tot_lodf_batches_N2)):
            #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
            lodf_batch = lodf_batch_size_N2
            if b == (tot_lodf_batches_N2 - 1): 
                lodf_batch = last_batch_size_N2
            left_ones = torch.ones((lodf_batch,1))
            right_ones = torch.ones((int(l*batch_size),1))
            Fli_0 = Flis.expand(lodf_batch, -1, -1)

            if b != (tot_lodf_batches_N2 - 1): 
                start_prob = b*lodf_batch*l
                end_prob = (b+1)*lodf_batch*l
                P_outage = probabilities_N2[start_prob:end_prob]
            if b == (tot_lodf_batches_N2 - 1): 
                end_prob = Ncontingencies_N2*l
                start_prob = (Ncontingencies_N2*l) - (last_batch_size_N2*l)
                P_outage = probabilities_N2[-(last_batch_size_N2*l):]

            index1 = LODF_dict_N2[b][0].detach()
            index2 = LODF_dict_N2[b][1].detach()
            indices = torch.stack((index1,index2)).detach()
            values = LODF_dict_N2[b][2].detach()
            shape = (l,int(l*lodf_batch))
            lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

            imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, Nsamples))) - max_f)).detach().to(device)
            #imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)
            #expected_violation = torch.mul(P_outage,imbalancetrain_batch_loop).to(torch.float32).reshape((lodf_batch, l, Nsamples))
     
            #======= get indices of contingencies of highest violations
            summed_tensor = torch.sum(torch.sum(imbalancetrain_batch_loop, dim=1), dim=1)

            if b == 0:
                desired_length = len(summed_tensor)

            padding_length = desired_length - len(summed_tensor)
            padded_tensor = F.pad(summed_tensor, (0, padding_length), value=0)
            imbalance_list2.extend(padded_tensor)


        indices_per_iteration = 20 # len(summed_tensor)

        stacked_imbalance = torch.stack(imbalance_list2, dim=0)
        highest_indices = (np.argsort(stacked_imbalance.cpu())[-indices_per_iteration:]).tolist()
        new_indices = [idx for idx in highest_indices if idx not in l2contingencies] # Filter out the indices that are already in the list
        l2contingencies.extend(new_indices) # Append the new indices to the list

    
time_scopfs = time.time() - time_scopfs
print('contingency screening time:', time_scopfs)

In [ ]:
#============== infeasible n-1 contingency cases
infeasibility_cont = lam1sc + lam2sc
infeasibility_cont_N1 = infeasibility_cont[:,0:31,:]
N1_counter = 0

for i in range(len(infeasibility_cont_N1[0,:,0])):
    intermediate = infeasibility_cont_N1[:,i,:]
    intermediate[abs(intermediate) < 1e-5] = 0
    intermediate_buses = (abs(intermediate) > 0) 
    intermediate_cases = torch.zeros((Nsamples,1))
    row_has_true_inter = torch.any(intermediate_buses, axis=0)
    intermediate_cases[row_has_true_inter] = True
    N1_counter += intermediate_cases.sum()
    

print(N1_counter)

In [ ]:
##============= check infeasibility
infeasibility_base = (lam1s + lam2s).permute(1,0)
infeasibility_base[abs(infeasibility_base) < 1e-5] = 0
infeasible_buses_base = (abs(infeasibility_base) > 1e-5) #0.001*load_profile) # 10% of total load available for generation/load shedding
scopf_cost_average = cost_scopf.mean()

infeasible_cases_base = torch.zeros((Nsamples,1))
row_has_true_base = torch.any(infeasible_buses_base, axis=1)
infeasible_cases_base[row_has_true_base] = True
infeasible_cases_tot_base = infeasible_cases_base.sum()
print('infeasible base cases:', infeasible_cases_tot_base)
print('percentage base infeasibility:', (infeasible_cases_tot_base/(Nsamples))*100, '%')
print('total average cost:', scopf_cost_average)

print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

In [ ]:
Flis = Fl0sscopf

imbalancetrain_batch_N1 = 0
imbalancetrain_batch_N2 = 0
imbalancetrain_batch_N3 = 0
imbalancetrain_batch = 0            

count_crit_violation_N1 = 0
count_line_violation_N1 = 0
count_violation_N1 = 0

count_crit_violation_N2 = 0
count_line_violation_N2 = 0
count_violation_N2 = 0

count_crit_violation_N3 = 0
count_line_violation_N3 = 0
count_violation_N3 = 0

expected_flows = torch.zeros(size = (l,Nsamples),dtype = torch.float32, requires_grad = True)
expected_flows_N1 = torch.zeros(size = (l,Nsamples),dtype = torch.float32, requires_grad = True)
expected_flows_N2 = torch.zeros(size = (l,Nsamples),dtype = torch.float32, requires_grad = True)
expected_flows_N3 = torch.zeros(size = (l,Nsamples),dtype = torch.float32, requires_grad = True)

#============ N-1
for b in (range(tot_lodf_batches_N1)):
    #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    lodf_batch = lodf_batch_size_N1
    if b == (tot_lodf_batches_N1 - 1): 
        lodf_batch = last_batch_size_N1
    left_ones = torch.ones((lodf_batch,1))
    right_ones = torch.ones((int(l*batch_size),1))
    Fli_0 = Flis.expand(lodf_batch, -1, -1)

    if b != (tot_lodf_batches_N1 - 1): 
        start_prob = b*lodf_batch*l
        end_prob = (b+1)*lodf_batch*l
        P_outage = probabilities_N1[start_prob:end_prob]
    if b == (tot_lodf_batches_N1 - 1): 
        end_prob = Ncontingencies_N1*l
        start_prob = (Ncontingencies_N1*l) - (last_batch_size_N1*l)
        P_outage = probabilities_N1[-(last_batch_size_N1*l):]

    index1 = LODF_dict_N1[b][0].detach()
    index2 = LODF_dict_N1[b][1].detach()
    indices = torch.stack((index1,index2)).detach()
    values = LODF_dict_N1[b][2].detach()
    shape = (l,int(l*lodf_batch))
    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, Nsamples))) - 1.1*max_f)).detach().to(device)
    count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
    count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)   
    LOLP_N1 = torch.where(count_line_violation_loop != 0, torch.tensor(1, dtype=count_line_violation_loop.dtype), count_line_violation_loop)
    LOLP_N1 = torch.mul(P_outage[0:Ncontingencies_N1],LOLP_N1)
    
    imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)

    #========= count the number of rows that have all zeros
    mask = torch.all(imbalancetrain_batch_loop == 0, dim=1) 
    indices_remove = mask.nonzero().squeeze(1)
    num_zero_rows = torch.sum(mask)

    indices_tot = torch.arange(start_prob, end_prob)
    mask_keep = ~torch.isin(indices_tot, indices_remove)
    indices_keep = indices_tot[mask_keep]
    indices_loc = indices_keep % l

    #============ create new lodf
    mask_flow = torch.ones(imbalancetrain_batch_loop.shape[0], dtype=torch.bool).to(device)
    mask_flow[indices_remove] = False
    lodf_test = lodf.to_dense()[mask_flow]
    lodf_test = lodf_test.clone().detach()
    lodf = lodf_test.to_sparse_coo().requires_grad_().to(torch.float32) 
    P_outage = P_outage[mask_flow]

    Fli_0 = Flis.repeat(lodf_batch, 1).to(torch.float32).to(device)[mask_flow]
    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0+torch.sparse.mm(lodf,Flis))- max_f))
    count_violation_loop = torch.count_nonzero(imbalancetrain_batch_loop)#/int(batch_size) 
    expected_violation = torch.mul(P_outage,imbalancetrain_batch_loop).to(torch.float32)

    imbalancetrain_batch_N1 += torch.sum(imbalancetrain_batch_loop)/int(Nsamples)
    expected_flows_N1 = torch.index_add(expected_flows_N1, 0, indices_loc.long(), expected_violation)

    count_violation_N1 += count_violation_loop
    count_crit_violation_N1 += torch.count_nonzero(count_crit_violation_loop.detach())
    count_line_violation_N1 += torch.count_nonzero(count_line_violation_loop.detach())

#============ N-2
for b in (range(tot_lodf_batches_N2)):
    #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    lodf_batch = lodf_batch_size_N2
    if b == (tot_lodf_batches_N2 - 1): 
        lodf_batch = last_batch_size_N2
    left_ones = torch.ones((lodf_batch,1))
    right_ones = torch.ones((int(l*batch_size),1))
    Fli_0 = Flis.expand(lodf_batch, -1, -1)

    if b != (tot_lodf_batches_N2 - 1): 
        start_prob = b*lodf_batch*l
        end_prob = (b+1)*lodf_batch*l
        P_outage = probabilities_N2[start_prob:end_prob]
    if b == (tot_lodf_batches_N2 - 1): 
        end_prob = Ncontingencies_N2*l
        start_prob = (Ncontingencies_N2*l) - (last_batch_size_N2*l)
        P_outage = probabilities_N2[-(last_batch_size_N2*l):]

    index1 = LODF_dict_N2[b][0].detach()
    index2 = LODF_dict_N2[b][1].detach()
    indices = torch.stack((index1,index2)).detach()
    values = LODF_dict_N2[b][2].detach()
    shape = (l,int(l*lodf_batch))
    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, Nsamples))) - 1.1*max_f)).detach().to(device)
    count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
    count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
    LOLP_N2 = torch.where(count_line_violation_loop != 0, torch.tensor(1, dtype=count_line_violation_loop.dtype), count_line_violation_loop)
    LOLP_N2 = torch.mul(P_outage[0:Ncontingencies_N2],LOLP_N2)
    
    imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)

    #========= count the number of rows that have all zeros
    mask = torch.all(imbalancetrain_batch_loop == 0, dim=1) 
    indices_remove = mask.nonzero().squeeze(1)
    num_zero_rows = torch.sum(mask)

    indices_tot = torch.arange(start_prob, end_prob)
    mask_keep = ~torch.isin(indices_tot, indices_remove)
    indices_keep = indices_tot[mask_keep]
    indices_loc = indices_keep % l

    #============ create new lodf
    mask_flow = torch.ones(imbalancetrain_batch_loop.shape[0], dtype=torch.bool).to(device)
    mask_flow[indices_remove] = False
    lodf_test = lodf.to_dense()[mask_flow]
    lodf_test = lodf_test.clone().detach()
    lodf = lodf_test.to_sparse_coo().requires_grad_().to(torch.float32) 
    P_outage = P_outage[mask_flow]

    Fli_0 = Flis.repeat(lodf_batch, 1).to(torch.float32).to(device)[mask_flow]
    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0+torch.sparse.mm(lodf,Flis))- max_f))
    count_violation_loop = torch.count_nonzero(imbalancetrain_batch_loop)#/int(batch_size) 
    expected_violation = torch.mul(P_outage,imbalancetrain_batch_loop).to(torch.float32)

    imbalancetrain_batch_N2 += torch.sum(imbalancetrain_batch_loop)/int(Nsamples)
    expected_flows_N2 = torch.index_add(expected_flows_N2, 0, indices_loc.long(), expected_violation)

    count_violation_N2 += count_violation_loop
    count_crit_violation_N2 += torch.count_nonzero(count_crit_violation_loop.detach())
    count_line_violation_N2 += torch.count_nonzero(count_line_violation_loop.detach())


#============ N-3
for b in (range(tot_lodf_batches_N3)):
    #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
    lodf_batch = lodf_batch_size_N3
    if b == (tot_lodf_batches_N3 - 1): 
        lodf_batch = last_batch_size_N3
    left_ones = torch.ones((lodf_batch,1))
    right_ones = torch.ones((int(l*batch_size),1))
    Fli_0 = Flis.expand(lodf_batch, -1, -1)

    if b != (tot_lodf_batches_N3 - 1): 
        start_prob = b*lodf_batch*l
        end_prob = (b+1)*lodf_batch*l
        P_outage = probabilities_N3[start_prob:end_prob]
    if b == (tot_lodf_batches_N3 - 1): 
        end_prob = Ncontingencies_N3*l
        start_prob = (Ncontingencies_N3*l) - (last_batch_size_N3*l)
        P_outage = probabilities_N3[-(last_batch_size_N3*l):]

    index1 = LODF_dict_N3[b][0].detach()
    index2 = LODF_dict_N3[b][1].detach()
    indices = torch.stack((index1,index2)).detach()
    values = LODF_dict_N3[b][2].detach()
    shape = (l,int(l*lodf_batch))
    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, Nsamples))) - 1.1*max_f)).detach().to(device)
    count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
    count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
    LOLP_N3 = torch.where(count_line_violation_loop != 0, torch.tensor(1, dtype=count_line_violation_loop.dtype), count_line_violation_loop)
    LOLP_N3 = torch.mul(P_outage[0:Ncontingencies_N3],LOLP_N3)
    
    imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)

    #========= count the number of rows that have all zeros
    mask = torch.all(imbalancetrain_batch_loop == 0, dim=1) 
    indices_remove = mask.nonzero().squeeze(1)
    num_zero_rows = torch.sum(mask)

    indices_tot = torch.arange(start_prob, end_prob)
    mask_keep = ~torch.isin(indices_tot, indices_remove)
    indices_keep = indices_tot[mask_keep]
    indices_loc = indices_keep % l

    #============ create new lodf
    mask_flow = torch.ones(imbalancetrain_batch_loop.shape[0], dtype=torch.bool).to(device)
    mask_flow[indices_remove] = False
    lodf_test = lodf.to_dense()[mask_flow]
    lodf_test = lodf_test.clone().detach()
    lodf = lodf_test.to_sparse_coo().requires_grad_().to(torch.float32) 
    P_outage = P_outage[mask_flow]

    Fli_0 = Flis.repeat(lodf_batch, 1).to(torch.float32).to(device)[mask_flow]
    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0+torch.sparse.mm(lodf,Flis))- max_f))

    count_violation_loop = torch.count_nonzero(imbalancetrain_batch_loop)#/int(batch_size) 
    expected_violation = torch.mul(P_outage,imbalancetrain_batch_loop).to(torch.float32)

    imbalancetrain_batch_N3 += torch.sum(imbalancetrain_batch_loop)/int(Nsamples)
    expected_flows_N3 = torch.index_add(expected_flows_N3, 0, indices_loc.long(), expected_violation)

    count_violation_N3 += count_violation_loop
    count_crit_violation_N3 += torch.count_nonzero(count_crit_violation_loop.detach())
    count_line_violation_N3 += torch.count_nonzero(count_line_violation_loop.detach())

expected_inf_N1 = torch.sum(expected_flows_N1)/int(Nsamples)
expected_inf_N2 = torch.sum(expected_flows_N2)/int(Nsamples)
expected_inf_N3 = torch.sum(expected_flows_N3)/int(Nsamples)

expected_infeasible_N1 = (torch.count_nonzero(expected_flows_N1)/torch.numel(expected_flows_N1))*100
expected_infeasible_N2 = (torch.count_nonzero(expected_flows_N2)/torch.numel(expected_flows_N2))*100
expected_infeasible_N3 = (torch.count_nonzero(expected_flows_N3)/torch.numel(expected_flows_N3))*100

print('')
#print('N1 infeasible contingency cases:', count_line_violation_N1, (count_line_violation_N1/(Ncontingencies_N1*Nsamples))*100, '%')
print('LOLP N1:', count_crit_violation_N1, (count_crit_violation_N1/(Ncontingencies_N1*Nsamples))*100, '%')
print('LOLP N1:', LOLP_N1.sum()/Nsamples)
print('LOLE h/y N1:', LOLP_N1.sum()/Nsamples*8760)
print('EENS N1:', expected_inf_N1*Sbase*frequency*repair_time*line_avg)
#print('N1 expected avg violation:', expected_inf_N1)
#print('N1 number violated lines:', expected_infeasible_N1)

print('')
print('N2 infeasible contingency cases:', count_line_violation_N2, (count_line_violation_N2/(Ncontingencies_N2*Nsamples))*100, '%')
print('LOLP N2:', count_crit_violation_N2, (count_crit_violation_N2/(Ncontingencies_N2*Nsamples))*100, '%')
print('LOLP N2:', LOLP_N2.sum()/Nsamples)
print('LOLE h/y N2:', LOLP_N2.sum()/Nsamples*8760)
print('EENS N2:', expected_inf_N2*Sbase*frequency*repair_time*2*line_avg*2)
#print('N2 expected avg violation:', expected_inf_N2)
#print('N2 number violated lines:', expected_infeasible_N2)

print('')
print('N3 infeasible contingency cases:', count_line_violation_N3, (count_line_violation_N3/(Ncontingencies_N3*Nsamples))*100, '%')
#print('LOLP N3:', count_crit_violation_N3, (count_crit_violation_N3/(Ncontingencies_N3*Nsamples))*100, '%')
print('LOLP N3:', LOLP_N3.sum()/Nsamples)
print('LOLE h/y N3:', LOLP_N3.sum()/Nsamples*8760)
print('EENS N3 MWh/y:', expected_inf_N3*Sbase*frequency*repair_time*3*line_avg*3)
#print('N3 expected avg violation:', expected_inf_N3)
#print('N3 number violated lines:', expected_infeasible_N3)

In [ ]:
start_test = time.time()
#============== Initialize training testing data
testing_dataset = Data.TensorDataset(Xscal, cost_scopf)
test_loader = Data.DataLoader(dataset=testing_dataset, batch_size=batch_size, shuffle=False)  # Note: shuffle=False for testing

#================ Initialize tensors
error = np.zeros(shape=(Niterations,3))
exp_inf = np.zeros(shape=(Niterations,3))
avg_infeasibility = np.zeros(shape=(Niterations,1))
relcost = np.zeros(shape=(Niterations,1))
cost = np.zeros(shape=(Niterations,1))

Pgis = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
Flis = torch.zeros(size = (Nlines,int(batch_size)),dtype = torch.float32)
clil = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
infeasibility = torch.zeros(int(batch_size), requires_grad = False)

relcotest_epoch = torch.zeros(size = (int(Nsamples/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)
cost_epoch = torch.zeros(size = (int(Nsamples/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)

imbalancetest_epoch = torch.zeros(size = (round((Nsamples)/batch_size),3),dtype = torch.float32, requires_grad = False)
expected_viol_test_epoch = torch.zeros(size = (round((Nsamples)/batch_size),3),dtype = torch.float32, requires_grad = False)

infeasibilitytest_epoch = torch.zeros(size = (int(Nsamples/batch_size),Nbus),dtype = torch.float32, requires_grad = False)
infeasibility_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

#===== N-1
crit_violation_count_N1 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
line_violation_count_N1 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

#===== N-2
crit_violation_count_N2 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
line_violation_count_N2 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

#===== N-3
crit_violation_count_N3 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
line_violation_count_N3 = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

print("It |   Time   |    Imbalance    | Mod out  |   Infeasibility   |  Rel cost")
for epoch in range(1):    
    with torch.no_grad():
        for i, (Xbatch, cost_scopf_batch) in enumerate(tqdm(test_loader)):
                
                model.eval() # deactivate dropout, when I changed this, testing error stayed same

                Xbatch = Xbatch.clone().detach()
                cost_scopf_batch = cost_scopf_batch.clone().detach()

                Xtest_batch = ((Xbatch.cpu() * Xstd)+Xmean).to(torch.float32).to(device)
                load_profile = torch.zeros((batch_size,Nbus)).to(device)
                load_profile[:, load_loc] = Xtest_batch

                #================== Perform forward pass
                #clil = (model(Xbatch))
                clil = (model(Xbatch)+1)/2
                
                #Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtest_batch, gencost, clil, solver_args=ECOS_solver_args)
                Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtest_batch, clil, solver_args=ECOS_solver_args)
                Pgis = Pgi.T
                Flis = Fli.T.to(torch.float32)
                
                load_profile[:, gen_loc] += Pgi
                nonzero_values, _ = torch.min(load_profile.masked_fill(load_profile == 0, float('inf')), dim=1, keepdim=True)
                mask = load_profile == 0
                load_profile = torch.where(mask, nonzero_values, load_profile)

                #================ compute cost and infeasibility
                infeasibility_batch = sum(lam1i.cpu().detach().numpy()) + sum(lam2i.cpu().detach().numpy())
                infeasibility_batch = torch.tensor(infeasibility_batch)
                infeasibility_tensor = lam1i + lam2i
                infeasibility_tensor[abs(infeasibility_tensor) < 1e-5 ] = 0
                infeasible_buses = (abs(infeasibility_tensor) > 1e-5)
                
                infeasible_cases = torch.zeros((batch_size,1))
                row_has_true = torch.any(infeasible_buses, axis=1)
                infeasible_cases[row_has_true] = True
                infeasible_cases = infeasible_cases.sum()
                
                batch_cost = torch.matmul(gencost,Pgis).cpu()
                batch_relco = ((batch_cost-cost_scopf_batch)/cost_scopf_batch)*100

                #===================== compute contingency flows for each sample
                imbalancetrain_batch_N1 = 0
                imbalancetrain_batch_N2 = 0
                imbalancetrain_batch_N3 = 0
                imbalancetrain_batch = 0   

                count_crit_violation_N1 = 0
                count_line_violation_N1 = 0
                count_violation_N1 = 0
                LOLP_N1_tot = 0

                count_crit_violation_N2 = 0
                count_line_violation_N2 = 0
                count_violation_N2 = 0
                LOLP_N2_tot = 0

                count_crit_violation_N3 = 0
                count_line_violation_N3 = 0
                count_violation_N3 = 0
                LOLP_N3_tot = 0
                
                expected_flows = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = True)
                expected_flows_N1 = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = True)
                expected_flows_N2 = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = True)
                expected_flows_N3 = torch.zeros(size = (l,batch_size),dtype = torch.float32, requires_grad = True)

                #============ N-1
                for b in (range(tot_lodf_batches_N1)):
                    #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
                    lodf_batch = lodf_batch_size_N1
                    if b == (tot_lodf_batches_N1 - 1): 
                        lodf_batch = last_batch_size_N1
                    left_ones = torch.ones((lodf_batch,1))
                    right_ones = torch.ones((int(l*batch_size),1))
                    Fli_0 = Flis.expand(lodf_batch, -1, -1)

                    if b != (tot_lodf_batches_N1 - 1): 
                        start_prob = b*lodf_batch*l
                        end_prob = (b+1)*lodf_batch*l
                        P_outage = probabilities_N1[start_prob:end_prob]
                    if b == (tot_lodf_batches_N1 - 1): 
                        end_prob = Ncontingencies_N1*l
                        start_prob = (Ncontingencies_N1*l) - (last_batch_size_N1*l)
                        P_outage = probabilities_N1[-(last_batch_size_N1*l):]

                    index1 = LODF_dict_N1[b][0].detach()
                    index2 = LODF_dict_N1[b][1].detach()
                    indices = torch.stack((index1,index2)).detach()
                    values = LODF_dict_N1[b][2].detach()
                    shape = (l,int(l*lodf_batch))
                    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

                    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, batch_size))) - 1.1*max_f)).detach().to(device)
                    count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
                    count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
                    LOLP_N1 = torch.where(count_line_violation_loop != 0, torch.tensor(1, dtype=count_line_violation_loop.dtype), count_line_violation_loop)
                    LOLP_N1 = torch.mul(P_outage[0:Ncontingencies_N1],LOLP_N1)
                    LOLP_N1_tot += LOLP_N1.sum()/int(batch_size)
                    
                    imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)

                    #========= count the number of rows that have all zeros
                    mask = torch.all(imbalancetrain_batch_loop == 0, dim=1) 
                    indices_remove = mask.nonzero().squeeze(1)
                    num_zero_rows = torch.sum(mask)

                    indices_tot = torch.arange(start_prob, end_prob)
                    mask_keep = ~torch.isin(indices_tot, indices_remove)
                    indices_keep = indices_tot[mask_keep]
                    indices_loc = indices_keep % l

                    #============ create new lodf
                    mask_flow = torch.ones(imbalancetrain_batch_loop.shape[0], dtype=torch.bool).to(device)
                    mask_flow[indices_remove] = False
                    lodf_test = lodf.to_dense()[mask_flow]
                    lodf_test = lodf_test.clone().detach()
                    lodf = lodf_test.to_sparse_coo().requires_grad_().to(torch.float32) 
                    P_outage = P_outage[mask_flow]

                    Fli_0 = Flis.repeat(lodf_batch, 1).to(torch.float32).to(device)[mask_flow]
                    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0+torch.sparse.mm(lodf,Flis))- max_f))
                    count_violation_loop = torch.count_nonzero(imbalancetrain_batch_loop)#/int(batch_size) 
                    expected_violation = torch.mul(P_outage,imbalancetrain_batch_loop).to(torch.float32)

                    imbalancetrain_batch_N1 += torch.sum(imbalancetrain_batch_loop)/int(batch_size)
                    expected_flows_N1 = torch.index_add(expected_flows_N1, 0, indices_loc.long(), expected_violation)

                    count_violation_N1 += count_violation_loop
                    count_crit_violation_N1 += torch.count_nonzero(count_crit_violation_loop.detach())
                    count_line_violation_N1 += torch.count_nonzero(count_line_violation_loop.detach())

                #============ N-2
                for b in (range(tot_lodf_batches_N2)):
                    #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
                    lodf_batch = lodf_batch_size_N2
                    if b == (tot_lodf_batches_N2 - 1): 
                        lodf_batch = last_batch_size_N2
                    left_ones = torch.ones((lodf_batch,1))
                    right_ones = torch.ones((int(l*batch_size),1))
                    Fli_0 = Flis.expand(lodf_batch, -1, -1)

                    if b != (tot_lodf_batches_N2 - 1): 
                        start_prob = b*lodf_batch*l
                        end_prob = (b+1)*lodf_batch*l
                        P_outage = probabilities_N2[start_prob:end_prob]
                    if b == (tot_lodf_batches_N2 - 1): 
                        end_prob = Ncontingencies_N2*l
                        start_prob = (Ncontingencies_N2*l) - (last_batch_size_N2*l)
                        P_outage = probabilities_N2[-(last_batch_size_N2*l):]

                    index1 = LODF_dict_N2[b][0].detach()
                    index2 = LODF_dict_N2[b][1].detach()
                    indices = torch.stack((index1,index2)).detach()
                    values = LODF_dict_N2[b][2].detach()
                    shape = (l,int(l*lodf_batch))
                    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

                    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, batch_size))) - 1.1*max_f)).detach().to(device)
                    count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
                    count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
                    LOLP_N2 = torch.where(count_line_violation_loop != 0, torch.tensor(1, dtype=count_line_violation_loop.dtype), count_line_violation_loop)
                    LOLP_N2 = torch.mul(P_outage[0:Ncontingencies_N2],LOLP_N2)
                    LOLP_N2_tot += LOLP_N2.sum()/int(batch_size)
                    
                    imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)

                    #========= count the number of rows that have all zeros
                    mask = torch.all(imbalancetrain_batch_loop == 0, dim=1) 
                    indices_remove = mask.nonzero().squeeze(1)
                    num_zero_rows = torch.sum(mask)

                    indices_tot = torch.arange(start_prob, end_prob)
                    mask_keep = ~torch.isin(indices_tot, indices_remove)
                    indices_keep = indices_tot[mask_keep]
                    indices_loc = indices_keep % l

                    #============ create new lodf
                    mask_flow = torch.ones(imbalancetrain_batch_loop.shape[0], dtype=torch.bool).to(device)
                    mask_flow[indices_remove] = False
                    lodf_test = lodf.to_dense()[mask_flow]
                    lodf_test = lodf_test.clone().detach()
                    lodf = lodf_test.to_sparse_coo().requires_grad_().to(torch.float32) 
                    P_outage = P_outage[mask_flow]

                    Fli_0 = Flis.repeat(lodf_batch, 1).to(torch.float32).to(device)[mask_flow]
                    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0+torch.sparse.mm(lodf,Flis))- max_f))
                    count_violation_loop = torch.count_nonzero(imbalancetrain_batch_loop)#/int(batch_size) 
                    expected_violation = torch.mul(P_outage,imbalancetrain_batch_loop).to(torch.float32)

                    imbalancetrain_batch_N2 += torch.sum(imbalancetrain_batch_loop)/int(batch_size)
                    expected_flows_N2 = torch.index_add(expected_flows_N2, 0, indices_loc.long(), expected_violation)

                    count_violation_N2 += count_violation_loop
                    count_crit_violation_N2 += torch.count_nonzero(count_crit_violation_loop.detach())
                    count_line_violation_N2 += torch.count_nonzero(count_line_violation_loop.detach())


                #============ N-3
                for b in (range(tot_lodf_batches_N3)):
                    #print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)
                    lodf_batch = lodf_batch_size_N3
                    if b == (tot_lodf_batches_N3 - 1): 
                        lodf_batch = last_batch_size_N3
                    left_ones = torch.ones((lodf_batch,1))
                    right_ones = torch.ones((int(l*batch_size),1))
                    Fli_0 = Flis.expand(lodf_batch, -1, -1)

                    if b != (tot_lodf_batches_N3 - 1): 
                        start_prob = b*lodf_batch*l
                        end_prob = (b+1)*lodf_batch*l
                        P_outage = probabilities_N3[start_prob:end_prob]
                    if b == (tot_lodf_batches_N3 - 1): 
                        end_prob = Ncontingencies_N3*l
                        start_prob = (Ncontingencies_N3*l) - (last_batch_size_N3*l)
                        P_outage = probabilities_N3[-(last_batch_size_N3*l):]

                    index1 = LODF_dict_N3[b][0].detach()
                    index2 = LODF_dict_N3[b][1].detach()
                    indices = torch.stack((index1,index2)).detach()
                    values = LODF_dict_N3[b][2].detach()
                    shape = (l,int(l*lodf_batch))
                    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

                    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf, Flis).reshape((lodf_batch, l, batch_size))) - 1.1*max_f)).detach().to(device)
                    count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
                    count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)
                    LOLP_N3 = torch.where(count_line_violation_loop != 0, torch.tensor(1, dtype=count_line_violation_loop.dtype), count_line_violation_loop)
                    LOLP_N3 = torch.mul(P_outage[0:Ncontingencies_N3],LOLP_N3)
                    LOLP_N3_tot += LOLP_N3.sum()/int(batch_size)
                    
                    imbalancetrain_batch_loop = imbalancetrain_batch_loop.reshape(int(l * lodf_batch), -1)

                    #========= count the number of rows that have all zeros
                    mask = torch.all(imbalancetrain_batch_loop == 0, dim=1) 
                    indices_remove = mask.nonzero().squeeze(1)
                    num_zero_rows = torch.sum(mask)

                    indices_tot = torch.arange(start_prob, end_prob)
                    mask_keep = ~torch.isin(indices_tot, indices_remove)
                    indices_keep = indices_tot[mask_keep]
                    indices_loc = indices_keep % l

                    #============ create new lodf
                    mask_flow = torch.ones(imbalancetrain_batch_loop.shape[0], dtype=torch.bool).to(device)
                    mask_flow[indices_remove] = False
                    lodf_test = lodf.to_dense()[mask_flow]
                    lodf_test = lodf_test.clone().detach()
                    lodf = lodf_test.to_sparse_coo().requires_grad_().to(torch.float32) 
                    P_outage = P_outage[mask_flow]

                    Fli_0 = Flis.repeat(lodf_batch, 1).to(torch.float32).to(device)[mask_flow]
                    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0+torch.sparse.mm(lodf,Flis))- max_f))

                    count_violation_loop = torch.count_nonzero(imbalancetrain_batch_loop)#/int(batch_size) 
                    expected_violation = torch.mul(P_outage,imbalancetrain_batch_loop).to(torch.float32)

                    imbalancetrain_batch_N3 += torch.sum(imbalancetrain_batch_loop)/int(batch_size)
                    expected_flows_N3 = torch.index_add(expected_flows_N3, 0, indices_loc.long(), expected_violation)

                    count_violation_N3 += count_violation_loop
                    count_crit_violation_N3 += torch.count_nonzero(count_crit_violation_loop.detach())
                    count_line_violation_N3 += torch.count_nonzero(count_line_violation_loop.detach())

                expected_inf_N1 = torch.sum(expected_flows_N1)/int(batch_size)
                expected_inf_N2 = torch.sum(expected_flows_N2)/int(batch_size)
                expected_inf_N3 = torch.sum(expected_flows_N3)/int(batch_size)

                expected_infeasible_N1 = (torch.count_nonzero(expected_flows_N1)/torch.numel(expected_flows_N1))*100
                expected_infeasible_N2 = (torch.count_nonzero(expected_flows_N2)/torch.numel(expected_flows_N2))*100
                expected_infeasible_N3 = (torch.count_nonzero(expected_flows_N3)/torch.numel(expected_flows_N3))*100
                torch.set_printoptions(precision=9)
                    
                imbalancetest_batch = expected_inf_N1+expected_inf_N2+expected_inf_N3
        
                relcotest_epoch[i,:] = batch_relco.detach()
                cost_epoch[i,:] = batch_cost.detach()
                imbalancetest_epoch[i,:] = torch.tensor([expected_inf_N1.detach(),expected_inf_N2.detach(),expected_inf_N3.detach()])
                expected_viol_test_epoch[i,:] = torch.tensor([expected_infeasible_N1.detach(),expected_infeasible_N2.detach(),expected_infeasible_N3.detach()])
                
                infeasibilitytest_epoch[i,:] = infeasibility_batch.detach()
                
                infeasibility_count[i,:] = infeasible_cases.detach()
                
                #====== N-1
                crit_violation_count_N1[i,:] = count_crit_violation_N1.detach()
                line_violation_count_N1[i,:] = count_line_violation_N1.detach()

                #====== N-2
                crit_violation_count_N2[i,:] = count_crit_violation_N2.detach()
                line_violation_count_N2[i,:] = count_line_violation_N2.detach()

                #====== N-3
                crit_violation_count_N3[i,:] = count_crit_violation_N3.detach()
                line_violation_count_N3[i,:] = count_line_violation_N3.detach()
            

    imbalancetest = torch.mean(imbalancetest_epoch, dim=0).detach()
    expected_violations_test = torch.mean(expected_viol_test_epoch, dim=0).detach()

    infeasibilitytest = torch.mean(infeasibilitytest_epoch).detach()

    relcotest = torch.mean(relcotest_epoch).detach()
    costtest = torch.mean(cost_epoch).detach()
    
    #======== N-1
    crit_violation_tot_N1 = torch.sum(crit_violation_count_N1).detach()
    line_violation_tot_N1 = torch.sum(line_violation_count_N1).detach()
    
    #======== N-1
    crit_violation_tot_N2 = torch.sum(crit_violation_count_N2).detach()
    line_violation_tot_N2 = torch.sum(line_violation_count_N2).detach()
    
    #======== N-1
    crit_violation_tot_N3 = torch.sum(crit_violation_count_N3).detach()
    line_violation_tot_N3 = torch.sum(line_violation_count_N3).detach()
    
    #====== convergence criteria
    infeasible_cases_tot = torch.sum(infeasibility_count).detach()
    
    error[epoch,:] = imbalancetest.detach().numpy()
    exp_inf[epoch,:] = expected_violations_test.detach().numpy()
    avg_infeasibility[epoch,:] =  infeasibilitytest.detach().numpy()
    relcost[epoch,:] =  relcotest.detach().numpy()
    cost[epoch,:] =  costtest.detach().numpy()
    end_test = time.time()
    time_test = end_test-start_test

    print(epoch, " | ", round(time_test,0), " | ",np.round(error[epoch,:],3), " | ",np.round(exp_inf[epoch,:],3), " | ", np.round(avg_infeasibility[epoch,:],3), " | ", np.round(relcost[epoch,:], 3) )    

print('method time:', time_test)

print('current infeasibility:', (infeasible_cases_tot/Nsamples)*100, '%', 'goal:', 1, '%')
    
print('')
print('N1 infeasible contingency cases:', crit_violation_tot_N1, (crit_violation_tot_N1/(Ncontingencies_N1*Nsamples))*100, '%')
print('LOLP N1:', LOLP_N1_tot )
print('LOLE h/y N1:', LOLP_N1_tot*8760)
print('EENS N1:', error[epoch,0]*Sbase*frequency*repair_time*line_avg)

print('')
print('N2 infeasible contingency cases:', crit_violation_tot_N2, (crit_violation_tot_N2/(Ncontingencies_N2*Nsamples))*100, '%')
print('LOLP N2:', LOLP_N2_tot)
print('LOLE h/y N2:', LOLP_N2_tot*8760)
print('EENS N2:', error[epoch,1]*Sbase*frequency*repair_time*2*line_avg*2) # double repair time for doubble fault, also double line length

print('')
print('N3 infeasible contingency cases:', crit_violation_tot_N3, (crit_violation_tot_N3/(Ncontingencies_N3*Nsamples))*100, '%')
print('LOLP N3:', LOLP_N3_tot)
print('LOLE h/y N3:', LOLP_N3_tot*8760)
print('EENS N3:', error[epoch,2]*Sbase*frequency*repair_time*3*line_avg*3) # triple repair time for triple fault, also triple line length



print('total cost:', cost.sum())



In [ ]:
#============== Create new training data (with high variability from RES)
X = np.array(data['load']['p'])/Sbase*np.random.uniform(low=0.75, high=1.25, size = (Nsamples,Nloads))
Xtrain = torch.tensor(X[0:int(Nsamples),:],dtype = torch.float32)        
Xtrain_transpose = Xtrain.transpose(0,1)
#Xtest = torch.tensor(X[int(train*Nsamples):,:],dtype = torch.float64)
#standardise
Xmin, Xmax,Xmean,Xstd = np.min(X, axis = 0),np.max(X, axis = 0),np.mean(X, axis = 0),np.std(X, axis = 0)
Xscal = torch.tensor(( X - Xmean ) / Xstd, dtype=torch.float32)

In [ ]:
#=============== Benchmark #2 heuristic approach
if case == 'N-1':
    benchmark = 'SCOPF1screening'
    
#benchmark = 'SCOPF4screening'

if benchmark == 'SCOPF1screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),1)))
elif benchmark == 'SCOPF2screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),2)))
elif benchmark == 'SCOPF3screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),3)))
elif benchmark == 'SCOPF4screening':
    Nk_contingencies = np.array(list(itertools.combinations(range(Nlines),4)))

Pgisscopf = torch.zeros(Ngens,int(Nsamples),dtype = torch.float32)
Fl0sscopf = torch.zeros(Nlines,int(Nsamples),dtype = torch.float32)
#Flcs = torch.zeros(Nlines,len(Nk_contingencies),int(Nsamples),dtype = torch.float32)
lam1s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
lam2s = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)
#lam1sc = torch.zeros(Nbus,len(Nk_contingencies),int(Nsamples),dtype = torch.float32)
#lam2sc = torch.zeros(Nbus,len(Nk_contingencies),int(Nsamples),dtype = torch.float32)
Th0 = torch.zeros(Nbus,int(Nsamples),dtype = torch.float32)

time_scopfs = time.time()

#lcontingencies = [0, 18426, 26007, 26009, 6858, 18433, 2363, 26011, 23394, 23371, 25970, 26010, 26006, 20773, 14570, 26046, 6856, 17846, 2361, 20771, 26005, 11034, 11052, 10820, 10785, 10780, 523, 5073, 10610, 5085, 10833, 11620, 10806, 11367, 10268, 680, 578, 10611, 590, 10995, 13086, 11874, 13096, 13114, 13113, 13108, 13112, 11390, 12274, 11643, 13107, 13109, 11114, 12084, 12854, 10814, 26013, 26014, 10667, 25995, 25996, 10916]
Nk_zeros = list(range(len(Nk_contingencies)))
zero_indices = [l for l in Nk_zeros if l not in lcontingencies]
print(lcontingencies)


if benchmark == 'SCOPF1screening':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf1_screening(data, lcontingencies)
elif benchmark == 'SCOPF2screening':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf2_screening(data, lcontingencies)
elif benchmark == 'SCOPF3screening':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf3_screening(data, lcontingencies)
elif benchmark == 'SCOPF4screening':
    problem0scopf, Pd0scopf, cg0scopf, Pgiscopf, Fl0scopf, Th0scopf, Flc, lam1, lam2, lam1c, lam2c = cs.create_scopf4_screening(data, lcontingencies)

cg0scopf.value = np.array(data['gen']['cost'])
for entry in tqdm(range(int(Nsamples)),position=0, leave=True):   
    Pd0scopf.value = X[entry,:] #np.array(data['load']['p'])/Sbase    
    solution  = problem0scopf.solve(solver=cp.ECOS)    # cp.MOSEK
    if problem0scopf.status in ["infeasible", "unbounded"]:
         # Otherwise, problem.value is inf or -inf, respectively.
         print("Problem infeasible or unbounded")
         #print("Optimal value: %s" % problem0scopf.value)
    Pgisscopf[:,entry] = torch.tensor(Pgiscopf.value)
    Fl0sscopf[:,entry] = torch.tensor(Fl0scopf.value)        
    Th0[:,entry] = torch.tensor(Th0scopf.value)

    lam1s[:,entry] = torch.tensor(lam1.value)
    lam2s[:,entry] = torch.tensor(lam2.value)

cost_scopf = torch.matmul(gencost,Pgisscopf)

#============ compute Nk flows
Flis = Fl0sscopf

for b in (range(tot_lodf_batches)):
    lodf_batch = lodf_batch_size
    if b == (tot_lodf_batches - 1): 
        lodf_batch = last_batch_size
    left_ones = torch.ones((lodf_batch,1))
    right_ones = torch.ones((int(l*batch_size),1))
    Fli_0 = Flis.expand(lodf_batch, -1, -1)

    index1 = LODF_dict[b][0].detach()
    index2 = LODF_dict[b][1].detach()
    indices = torch.stack((index1,index2)).detach()
    values = LODF_dict[b][2].detach()
    shape = (l,int(l*lodf_batch))
    lodf = torch.sparse_coo_tensor(indices,values,size = shape, requires_grad = False).T.to(device).detach()

    imbalancetrain_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf,Flis).reshape((lodf_batch,l,Nsamples)))- max_f)).detach()
    count_crit_violation_loop = torch.sum(imbalancetrain_batch_loop > tollerance_crit*max_f, dim=1)
    count_line_violation_loop = torch.sum(imbalancetrain_batch_loop > 0, dim=1)

    count_crit_violation += torch.count_nonzero(count_crit_violation_loop)
    count_line_violation += torch.count_nonzero(count_line_violation_loop)

crit_violation_tot = count_crit_violation
line_violation_tot = count_line_violation

print('infeasible N-k contingency cases:', line_violation_tot, (line_violation_tot/(Ncontingencies*Nsamples))*100, '%')
print('10%+ contingency cases:', crit_violation_tot, (crit_violation_tot/(Ncontingencies*Nsamples))*100, '%')
    
time_scopfs = time.time() - time_scopfs
print('heuristic approach time:', time_scopfs)

##============= check infeasibility
infeasibility_base = (lam1s + lam2s).permute(1,0)
infeasibility_base[abs(infeasibility_base) < 1e-4] = 0
infeasible_buses_base = (abs(infeasibility_base) > 1e-4) #0.001*load_profile) # 10% of total load available for generation/load shedding
scopf_cost_average = cost_scopf.mean()

infeasible_cases_base = torch.zeros((Nsamples,1))
row_has_true_base = torch.any(infeasible_buses_base, axis=1)
infeasible_cases_base[row_has_true_base] = True
infeasible_cases_tot_base = infeasible_cases_base.sum()
print('infeasible base cases:', infeasible_cases_tot_base)
print('percentage base infeasibility:', (infeasible_cases_tot_base/(Nsamples))*100, '%')
print('total average cost:', scopf_cost_average)

print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

In [ ]:
start_test = time.time()
#============== Initialize training testing data
testing_dataset = Data.TensorDataset(Xscal, cost_scopf)
test_loader = Data.DataLoader(dataset=testing_dataset, batch_size=batch_size, shuffle=False)  # Note: shuffle=False for testing

#================ Initialize tensors
error = np.zeros(shape=(Niterations,1))
avg_infeasibility = np.zeros(shape=(Niterations,1))
relcost = np.zeros(shape=(Niterations,1))
cost = np.zeros(shape=(Niterations,1))

Pgis = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
Flis = torch.zeros(size = (Nlines,int(batch_size)),dtype = torch.float32)
clil = torch.zeros(size = (Ngens,int(batch_size)),dtype = torch.float32)
infeasibility = torch.zeros(int(batch_size), requires_grad = False)

relcotest_epoch = torch.zeros(size = (int(Nsamples/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)
cost_epoch = torch.zeros(size = (int(Nsamples/batch_size),int(batch_size)),dtype = torch.float32, requires_grad = False)

imbalancetest_epoch = torch.zeros(size = (round((Nsamples)/batch_size),1),dtype = torch.float32, requires_grad = False)

infeasibilitytest_epoch = torch.zeros(size = (int(Nsamples/batch_size),Nbus),dtype = torch.float32, requires_grad = False)
infeasibility_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
line_violation_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
crit_violation_count = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
avg_violation_list = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)
count_violation_list = torch.zeros(size = (int(Nsamples/batch_size),1),dtype = torch.float32, requires_grad = False)

print("It |   Time   |    Imbalance    | Mod out  |   Infeasibility   |  Rel cost")
for epoch in range(1):    
    with torch.no_grad():
        for i, (Xbatch, cost_scopf_batch) in enumerate(tqdm(test_loader)):
                
                model.eval() # deactivate dropout, when I changed this, testing error stayed same

                Xbatch = Xbatch.clone().detach()
                cost_scopf_batch = cost_scopf_batch.clone().detach()

                Xtest_batch = ((Xbatch.cpu() * Xstd)+Xmean).to(torch.float32).to(device)
                load_profile = torch.zeros((batch_size,Nbus)).to(device)
                load_profile[:, load_loc] = Xtest_batch

                #================== Perform forward pass
                #clil = (model(Xbatch))
                clil = (model(Xbatch)+1)/2
                
                Pgi, Fli, Thi, lam1i, lam2i = cvxpylayer0(Xtest_batch, gencost, clil, solver_args=ECOS_solver_args)
                Pgis = Pgi.T
                Flis = Fli.T.to(torch.float32)
                
                load_profile[:, gen_loc] += Pgi
                nonzero_values, _ = torch.min(load_profile.masked_fill(load_profile == 0, float('inf')), dim=1, keepdim=True)
                mask = load_profile == 0
                load_profile = torch.where(mask, nonzero_values, load_profile)

                #================ compute cost and infeasibility
                infeasibility_batch = sum(lam1i.cpu().detach().numpy()) + sum(lam2i.cpu().detach().numpy())
                infeasibility_batch = torch.tensor(infeasibility_batch)
                infeasibility_tensor = lam1i + lam2i
                infeasibility_tensor[abs(infeasibility_tensor) < 1e-5 ] = 0
                infeasible_buses = (abs(infeasibility_tensor) > 1e-5)
                
                infeasible_cases = torch.zeros((batch_size,1))
                row_has_true = torch.any(infeasible_buses, axis=1)
                infeasible_cases[row_has_true] = True
                infeasible_cases = infeasible_cases.sum()
                
                batch_cost = torch.matmul(gencost,Pgis).cpu()
                batch_relco = ((batch_cost-cost_scopf_batch)/cost_scopf_batch)*100

                #===================== compute contingency flows for each sample
                imbalancetest_batch = 0
                count_crit_violation = 0
                count_line_violation = 0
                count_violation = 0

                for b in (range(tot_lodf_batches)):
                    lodf_batch = lodf_batch_size
                    if b == (tot_lodf_batches - 1): 
                        lodf_batch = last_batch_size
                    left_ones = torch.ones((lodf_batch_size,1))
                    right_ones = torch.ones((int(l*batch_size),1))
                    Fli_0 = Flis.expand(lodf_batch, -1, -1)#.contiguous().view(-1, len(Flis[1])).detach()#.to(torch.float32).clone()
                    #mask = mask_base(b, lodf_batch)
                    #Fli_0[~mask] = 0

                    index1 = LODF_dict[b][0].detach()
                    index2 = LODF_dict[b][1].detach()
                    indices = torch.stack((index1,index2)).detach()
                    values = LODF_dict[b][2].detach()
                    shape = (l,int(l*lodf_batch))
                    lodf = torch.sparse_coo_tensor(indices,values,size = shape).T.to(device)

                    imbalancetest_batch_loop = (F.relu(torch.abs(Fli_0 + torch.sparse.mm(lodf,Flis).reshape((lodf_batch,l,batch_size)))- max_f)).to(device)
                    imbalancetest_batch_loop[abs(imbalancetest_batch_loop) < 1e-5] = 0
                    count_crit_violation_loop = torch.sum(imbalancetest_batch_loop > tollerance_crit*max_f, dim=1)
                    count_line_violation_loop = torch.sum(imbalancetest_batch_loop > 0, dim=1)                   
                    
                    imbalancetest_batch_loop = imbalancetest_batch_loop.reshape(lodf_batch,int(l*batch_size)).to(device)
                    count_violation_loop = torch.count_nonzero(imbalancetest_batch_loop)
                
                    imbalancetest_batch += torch.sum(imbalancetest_batch_loop)/int(batch_size)
                    count_violation += count_violation_loop
                    count_crit_violation += torch.count_nonzero(count_crit_violation_loop.detach()).to(device)
                    count_line_violation += torch.count_nonzero(count_line_violation_loop.detach()).to(device)
        
                relcotest_epoch[i,:] = batch_relco.detach()
                cost_epoch[i,:] = batch_cost.detach()
                imbalancetest_epoch[i,:] = imbalancetest_batch.detach()
                infeasibilitytest_epoch[i,:] = infeasibility_batch.detach()
                
                infeasibility_count[i,:] = infeasible_cases.detach()
                line_violation_count[i,:] = count_line_violation.detach()
                crit_violation_count[i,:] = count_crit_violation.detach()
                #avg_violation_list[i,:] = avg_violation.detach()
                count_violation_list[i,:] = count_violation
            

    imbalancetest = torch.mean(imbalancetest_epoch).detach()

    infeasibilitytest = torch.mean(infeasibilitytest_epoch).detach()

    relcotest = torch.mean(relcotest_epoch).detach()
    costtest = torch.mean(cost_epoch).detach()
    
    #====== convergence criteria
    infeasible_cases_tot = torch.sum(infeasibility_count).detach()
    line_violation_tot = torch.sum(line_violation_count).detach()
    crit_violation_tot = torch.sum(crit_violation_count).detach()
    #tot_avg_violation = torch.sum(avg_violation_list).detach()
    count_violation_tot = torch.sum(count_violation_list).detach()
    tot_flows = l*Ncontingencies*batch_size
    
    error[epoch,:] = imbalancetest.detach().numpy()
    avg_infeasibility[epoch,:] =  infeasibilitytest.detach().numpy()
    relcost[epoch,:] =  relcotest.detach().numpy()
    cost[epoch,:] =  costtest.detach().numpy()
    end_test = time.time()
    time_test = end_test-start_test

    print(epoch, " | ", round(time_test,0), " | ",np.round(error[epoch,:],3), " | ", np.round(torch.mean(clil.cpu()).detach().numpy(),3), " | ", np.round(avg_infeasibility[epoch,:],3), " | ", np.round(relcost[epoch,:], 3) )
        

print('method time:', time_test)

print('current infeasibility:', (infeasible_cases_tot/Nsamples)*100, '%', 'goal:', 1, '%')
    
#print('current avg violation:', (tot_avg_violation/initial_violation_count),'pu', 'goal', 0.01*max_f, 'pu')
print('infeasible contingency cases:', line_violation_tot, (line_violation_tot/(Ncontingencies*Nsamples))*100, '%')
print('10%+ infeasible cases:', crit_violation_tot, (crit_violation_tot/(Ncontingencies*Nsamples))*100, '%')
print('number of violations:', count_violation_tot)
print('total cost:', cost.sum())